# Benchmark Model

## Functional code block

In [105]:
mutable struct MarkovChain
    markovchain::Array{Float64,2}
    state_values::Array{Float64,1}
    weights::Array{Float64,1}
end

In [106]:
mutable struct policy_variables
"""
    Xt: cash-on-hand at period t, endogenously determined
    Ct: consumption at period t
    Kt: sainvg at period t, endogenously determined
    Cpt: consumption at period t+1
    lt_egm: labor supply at time t over grid (Kt,a_vals)
    lt_grid: labor suppy at time t over grid a_vals
    lpt_grid: labor supply at time t+1 over grid _avals
    """
    
    Xt::Array{Float64,2}
    Kt::Array{Float64,2}
    Kppt::Array{Float64,2}
    lt_grid::Array{Float64,2}
    lpt_grid::Array{Float64,2}
    lpti_grid::Array{Float64,2}
end

In [107]:
"""
Stores all the parameters that define the household's
problem.

##### Fields


- `σ::Real` : risk aversion
- `β::AbstractFloat` : discount factor
- `α::TF : capital share
- `δ::TF : depreciation rate
- `ψ::TF : labor disutility
- `ζ::TF : Fisch labor elasticity
- `p_chain::MarkovChain` : MarkovChain for persistent income productivity
- `a_min::Real` : minimum on asset grid
- `a_max::Real` : maximum on asset grid
- `a_size::Integer` : number of points on asset grid
- `z_size::Integer` : number of points on income grid
- `z_size::Integer` : number of points on income grid
"""

mutable struct Household{TR<:Real,TF<:AbstractFloat, TI<:Integer}
    # 13 parameters that's invariant for household
    σ::TF
    β_chain::MarkovChain
    α::TF
    δ::TF
    ψ::TF
    ζ::TF
    a_min::TR
    a_max::TR
    a_size::TI
    a_vals::AbstractVector{TF}
    a_cdf_size::TI
    a_cdf_vals::AbstractVector{TF}
    u::Function
    λT::TF
end

In [108]:
mutable struct Tax
    # 6 variables
    y_pshare::Array{Float64,1}
    y_ishare::Array{Float64,1}
    y_atax::Array{Float64,1}
    y_mtax::Array{Float64,1}
    y_cutoff::Array{Float64,1}
    y_end_tax::Array{Float64,1}
end

In [109]:
mutable struct aggregate_variables
    #7 variables
    r::Float64
    w::Float64
    K::Float64
    L::Float64
    Transfer::Float64
    tax::Tax
    Gz::Array{Float64,2}
end

In [110]:
mutable struct productivity_variables
    # 9 variables
    pt::MarkovChain
    vt::MarkovChain
    p0::MarkovChain
    n1::Integer
    n2::Integer
    n3::Integer
    n4::Integer
    n5::Integer
    trans_matrix::Array{Float64,2}
end

In [111]:
mutable struct error_recording
    # 7 variables
    K::Float64
    L::Float64
    r::Float64
    Xt::Float64
    Kt::Float64
    Lpt::Float64
    Kppt::Float64
end

In [112]:
function vector_linear_interpolation(X1::Array{Float64,1},Y1::Array{Float64,1},X2::Array{Float64,1})
    """
    Given X1 mapping into Y1, linear interpolate on X2, with linear extrapoliation outside of range
    Require X1,Y1 same size
    """
    itp = LinearInterpolation(X1,Y1, extrapolation_bc = Line())
    Y2 = itp(X2)
    return Y2
end

vector_linear_interpolation (generic function with 1 method)

In [113]:
function productivity(;pt,vt,p0,β_chain)
""" Input:
    State value of pt,vt, β_chain and p0
    
    Output:
    supplements with transitional matrix and size of different combination of productivity states"""
    pt_size = length(pt.state_values)
    vt_size = length(vt.state_values)
    p0_size = length(p0.state_values)
    β_size = length(β_chain.state_values)
    
    trans_matrix = kron(kron(kron(pt.markovchain,β_chain.markovchain),vt.weights'),p0.weights')
    n1 = pt_size*β_size*vt_size*p0_size
    n2 = pt_size*β_size
    n3 = n2*vt_size
    prod = productivity_variables(pt,vt,p0,n1,n2,n3,p0_size*vt_size,p0_size,trans_matrix)
    return prod
end

productivity (generic function with 1 method)

In [114]:
function integration_matrix(;n_matrix,n_grid,weight)
""" Input: Original transitional matrix, unconditional weight on old grid and new grid
    Output: trantional matrix on new grid"""
    m,n = size(n_matrix)
    λ_matrix = zeros(m,m)
    for i = 1:m
        for j = 1:n
            if (n_matrix[i,j]<=maximum(n_grid)) && (n_matrix[i,j]>minimum(n_grid))
                I = findnext(n_grid.>n_matrix[i,j],1)
                lambda = (n_matrix[i,j]-n_grid[I-1])/(n_grid[I]-n_grid[I-1])
                λ_matrix[i,I-1] += (1-lambda)*weight[j]
                λ_matrix[i,I] += lambda*weight[j]
            elseif n_matrix[i,j]>maximum(n_grid)
                λ_matrix[i,end] += weight[j]
            else
                λ_matrix[i,1] += weight[j]
            end
        end
    end
    return λ_matrix
end

integration_matrix (generic function with 1 method)

In [115]:
function GaussHermite_matrix(;n::Integer,μ=0,rho,σ,normalize=1,exponential=1)
""" convert an AR1 process into Gauss-Hermite nodes with integration matrix
    nodes are related Gauss-Hermite integration nodes
   """ 
    σ_ϵ=σ*sqrt(1-rho^2)
    nodes,weight = gausshermite(n) 
    weight = weight./sqrt(pi)
    x_grid = sqrt(2)*σ*nodes.+μ
    
    #Pre-compute the intergration weight matrix
    x_matrix = Array{Float64,2}(undef,n,n)
    for i = 1: n
        x_matrix[:,i] = rho*x_grid.+sqrt(2)*σ_ϵ*nodes[i].+(1-rho)*μ
    end
    if exponential==1
        x_matrix = exp.(x_matrix)
        x_grid = exp.(x_grid)
    end
    λ_matrix = integration_matrix(n_matrix = x_matrix,n_grid = x_grid,weight = weight)
    # normalization such that E(exp(x))=1
    if normalize==1 && exponential==1
        N_weight = exp(σ^2/2)
        x_grid = x_grid/N_weight
    end
    return λ_matrix,x_grid,weight
end

GaussHermite_matrix (generic function with 1 method)

In [116]:
function pt_unconditional_distribution(;pt,φ,σ_pt)
    """
    Calculating the unconditional distribution of persistent productivity pt
    """
    NN = 100000

    PT = range(minimum(pt)-1,stop = maximum(pt)+1,length=NN)

    ipt_nnmatrix = LinearInterpolation(pt, φ, extrapolation_bc = Line())
    φ_NN_matrix = ipt_nnmatrix([p for p in PT])

    fp = pdf.(Normal(0,σ_pt),PT)
    λp_NN_matrix = zeros(length(pt))
    
    for i = 1:NN
        if PT[i]<=maximum(pt) && PT[i]>minimum(pt)
            I = findnext(pt.>PT[i],1)
            λ = (φ_NN_matrix[i]-φ[I-1])/(φ[I]-φ[I-1])
            λp_NN_matrix[I-1] = λp_NN_matrix[I-1]+(1-λ)*fp[i]
            λp_NN_matrix[I] = λp_NN_matrix[I]+λ*fp[i]

        elseif PT[i]>maximum(pt)
            λp_NN_matrix[end] = λp_NN_matrix[end]+fp[i]
        else
            λp_NN_matrix[1] = λp_NN_matrix[1] + fp[i]
        end
    end

    λp_NN_matrix = λp_NN_matrix./sum(λp_NN_matrix)
    return λp_NN_matrix
end

pt_unconditional_distribution (generic function with 1 method)

In [117]:
function ptmatrix(;n,ρ_pt,σ_pt,kt,shrink=1)
""" Input: Number of grid, AR1 ρ and σ, Pareto tail parameter kt and grid shrinking factor
    Output: transitional matrix, unconditional weights on specified grid, productivity level on new grid
"""
    perc_pt=[0.0001, 0.01, 0.1, 0.25, 0.5, 0.75, 0.9,
        0.925, 0.95, 0.975, 0.99, 0.999, 0.9999, 0.99999, 0.999999, 0.9999999, 0.99999999]
    m = length(perc_pt)
    pt = quantile.(Normal(0,σ_pt),[0.0001, 0.01, 0.1, 0.25, 0.5, 0.75, 0.9,
        0.925, 0.95, 0.975, 0.99, 0.999, 0.9999, 0.99999, 0.999999, 0.9999999, 0.99999999])
    xm = pt[7] # the threshold F(pt)=0.9
    φ_ptail = quantile.(Pareto(kt,xm),(perc_pt[8:end].-0.9)./(1-0.9))
    φ = exp.(pt)
    scale_ptail = φ[7]/quantile.(Pareto(kt,xm),0)
    φ[8:end] = φ_ptail*scale_ptail
    φ = exp.(log.(φ)./shrink)

    ###############################################################
    # Obtain the unconditional distribution
    λp_NN_matrix = pt_unconditional_distribution(pt=pt,φ=φ,σ_pt=σ_pt)
    ###############################################################
    φ = φ./(λp_NN_matrix'*φ)
    #Pre-compute the intergration weight matrix
    p_nodes,p_weight = gausshermite(n) 
    p_weight = p_weight./sum(p_weight)
    σ_pt_error = sqrt(1-ρ_pt^2)*σ_pt

    pn_matrix = zeros(m,n)
    for i =1:n
        pn_matrix[:,i] = ρ_pt.*pt.+sqrt(2)*σ_pt_error*p_nodes[i]
    end
    
    φ_n_matrix = Array{Float64,2}(undef,m,n)
    itp_matrix = LinearInterpolation(pt, φ, extrapolation_bc = Line())
    for i =1:n
        φ_n_matrix[:,i] = itp_matrix(pn_matrix[:,i])
    end
    λp_matrix = integration_matrix(n_matrix = φ_n_matrix,n_grid = φ,weight = p_weight)

    pt_chain = MarkovChain(λp_matrix,φ,λp_NN_matrix)
    return pt_chain
    
end

ptmatrix (generic function with 1 method)

In [118]:
function parameters(;
            σ::Float64,
            β_chain::MarkovChain,
            α::Float64,
            δ::Float64,
            ψ::Float64,
            ζ::Float64,
            a_min::Real=1e-10,
            a_max::Real,
            a_size::Integer,
            a_cdf_size::Integer,
            λT::Float64)
"""    Output: Obtain Household related parameters"""
    
    # set up the grids
    curv =0.25 
    a_vals = (range(0^curv,stop=(a_max-a_min)^curv,length=a_size)).^(1/curv)
    a_vals = a_vals.+a_min

    # Create asset grid for CDF
    a_cdf_vals = (range(0^curv,stop=(a_max-a_min)^curv,length=a_cdf_size)).^(1/curv)
    a_cdf_vals = a_cdf_vals.+a_min
    
    # utility function
    if σ==1
        u = x-> log(x)
    else
        u = x-> (x^(1-σ)-1)/(1-σ)
    end
    # 15 inputs
    h = Household(σ,β_chain,α,δ,ψ,ζ,a_min,a_max,a_size,a_vals,a_cdf_size,a_cdf_vals,u,λT)
    return h
end

parameters (generic function with 1 method)

In [119]:
function tax_policy(;Tax::Tax,r::Float64,w::Float64,k::Float64,L::Float64)
"""
    Calculate the cutoff point for each tax bracket related its corresponding population share and transfer level,
    given aggregate income
    """
    K = k*L
    yibar = r*K + w*L
    
    
    y_mtax = Array{Float64,1}(undef,11)
    y_cutoff = zeros(12)
    
    for i = 1:11
        y_cutoff[i+1] = 2*Tax.y_ishare[i]*yibar/Tax.y_pshare[i]-y_cutoff[i]
#         y_cutoff[i+1] = 2*Tax.y_ishare[i]/Tax.y_pshare[i]-y_cutoff[i]
    end
    
#         %%%%%%%%%%%%%%%
#     % set the marginal tax rate such that the average tax rate in each income
#     % group matches the data

#     % could be written as a system of linear equations solving 11 unknows(marginal tax rate for each income groups)
#     % with 11 functions: the average income tax rate has to match the data in
#     % Piketty and Saez(2007)
    # Create the coefficient matrix 11X11
    A = zeros(11,11)
    A[1,1] = (y_cutoff[2,1]-y_cutoff[1,1])/y_cutoff[2,1]
    
    
    # Diagonal Value
    for i =2:11
        A[i,i] = 1-y_cutoff[i,1]/(y_cutoff[i+1]-y_cutoff[i])*log(y_cutoff[i+1]/y_cutoff[i])
    end
    
    for i =2:11
        for j=1:(i-1)
            A[i,j] = (y_cutoff[j+1]-y_cutoff[j])/(y_cutoff[i+1]-y_cutoff[i])*log(y_cutoff[i+1]/y_cutoff[i])
        end
    end
    
    y_mtax[:] = A\Tax.y_atax
    y_end_tax=zeros(12)
    
    y_end_tax[1,1] = 0.0
    for i=2:12
        y_end_tax[i] = y_end_tax[i-1]+y_mtax[i-1]*(y_cutoff[i]-y_cutoff[i-1])
    end
    
    for i =1:11
        if y_cutoff[i]>y_cutoff[i+1]
            y_cutoff[i]=y_cutoff[i+1]-0.001
        end
    end
    itp_tax = LinearInterpolation(y_cutoff, y_end_tax, extrapolation_bc = Line())
    Totaltax = itp_tax(yibar)
    Transfer = λT*Totaltax
    
    Tax.y_mtax = y_mtax
    Tax.y_cutoff = y_cutoff
    Tax.y_end_tax = y_end_tax
    return Tax,Transfer,itp_tax
end


tax_policy (generic function with 1 method)

In [120]:
function marginaltax(;y,tax::Tax)
    """
    marginaltax function calculate the marginal tax rate for given
    INPUT:
    y: a vecotr/matrix of income y to be evaluated
    ycutoff: 12 cutoff points for 11 income groups
    ymtax: marginal tax rate for each income groups

    OUTPUT:
    ymt: a vector/matrix of marginal tax rate for income y
   """
    a,b = size(y)
    
    ymt = zeros(a,b)
    ymt[y.>maximum(tax.y_cutoff)].=tax.y_mtax[end]
    ymt[y.<minimum(tax.y_cutoff)].=0.0
    
    for i = 1:11
        Indi = (y.>tax.y_cutoff[i]).*(y.<=tax.y_cutoff[i+1])
        ymt[Indi] .= tax.y_mtax[i]
    end
    return ymt
end

marginaltax (generic function with 1 method)

In [121]:
function op_labor1(;w::Array{Float64,1},r::Float64,hh::Household,
        kprime::Array{Float64,2},k_grid::Array{Float64,2},itp_tax,Transfer::Float64,tax::Tax)
"""
    Solve labor optimality condition using RHS of Euler equation for t and t+1
    be careful of the period t of RHS when using.
    """
    x1,x2 = size(kprime)
    la = zeros(x1,x2).+0.001
    lb = ones(x1,x2)
    # labor disutility - benefit of labor supplying
    
    Ya = r*k_grid+broadcast(*,la,w')
    Ya_tax = Array{Float64,2}(undef,x1,x2)
    for j = 1:x2
        Ya_tax[:,j] = itp_tax(Ya[:,j])
    end
    
    Ya_tax[Ya_tax.<0.0].=0.0
    Ya_tau = marginaltax(y=Ya,tax=tax) 
    
    Yb = r*k_grid+broadcast(*,lb,w')
    Yb_tax = Array{Float64,2}(undef,x1,x2)
    for j = 1:x2
        Yb_tax[:,j] = itp_tax(Yb[:,j])
    end
    
    Yb_tax[Yb_tax.<0.0].=0.0
    Yb_tau = marginaltax(y=Yb,tax=tax)
    
    Cta =k_grid+Ya-Ya_tax-kprime.+Transfer
    Cta[Cta.<=0.0].=0.0001
    fla = hh.ψ*la.^hh.ζ-(-Ya_tau.+1).*broadcast(*,(Cta).^(-hh.σ),w')
    
    
    Ctb =k_grid+Yb-Yb_tax-kprime.+Transfer
    Ctb[Ctb.<=0.0].=0.0001    
    flb = hh.ψ*lb.^hh.ζ-(-Yb_tau.+1).*broadcast(*,(Ctb).^(-hh.σ),w')

    dlx = 0.5*(lb-la)
    lx = la+dlx
    tol=1e-6
    iter = 0
    C_min = repeat(range(1,length=x1).*1e-6,1,x2)
    Yx_tax = Array{Float64,2}(undef,x1,x2)
    while any(abs.(dlx).>tol)
        iter+=1
        dlx = 0.5*dlx
        Yx = r*k_grid+broadcast(*,lx,w')
        for j =1:x2
            Yx_tax[:,j] = itp_tax(Yx[:,j])
        end
        Yx_tax[Yx_tax.<0.0].=0.0
        Yx_tau = marginaltax(y=Yx,tax=tax)
        Ct = k_grid+Yx-Yx_tax-kprime.+Transfer
#         Ct[Ct.<=0.0].=C_min[Ct.<=0.0]
        Ct[Ct.<=0.0].=0.0001
        flx = hh.ψ*lx.^hh.ζ-(-Yx_tau.+1).*broadcast(*,(Ct).^(-hh.σ),w')
        lx = lx -sign.(flx).*dlx
    end
    
    # there are cases that FOC on labor not balanced
    # the benefit of working always surpasses the cost of working
    lx[lx.>1.0].=1.0
    lx[lx.<0.0].=0.0
#     lx[sign.(flb).<0].=1
    return lx
end

op_labor1 (generic function with 1 method)

In [122]:
function op_labor2(;r::Float64,w::Array{Float64,1},hh::Household,
        k_grid::Array{Float64,2},tax::Tax,RHS::Array{Float64,2},itp_tax)
"""
    Solve labor optimality condition using RHS of Euler equation for t and t+1
    be careful of the period t of RHS when using.
    """
    x1,x2 = size(k_grid)
    la = zeros(x1,x2).+0.001
    lb = ones(x1,x2)
    # labor disutility - benefit of labor supplying
    Ya = k_grid.*r+broadcast(*,la,w')
    Yatau = marginaltax(y=Ya,tax=tax)
    
    Yb = k_grid.*r+broadcast(*,lb,w')
    Ybtau = marginaltax(y=Yb,tax=tax)
    
    fla = hh.ψ*la.^hh.ζ-(-Yatau.+1).*broadcast(*,RHS,w')
    flb = hh.ψ*lb.^hh.ζ-(-Ybtau.+1).*broadcast(*,RHS,w')
    
    dlx = 0.5*(lb-la)
    lx = la+dlx
    tol=1e-4
    iter = 0
    C_min = repeat(range(1,length=x1).*1e-6,1,x2)
    while any(abs.(dlx).>tol)
        iter+=1
        dlx = 0.5*dlx
        Yx = k_grid.*r+broadcast(*,lx,w')
        Yxtau = marginaltax(y=Yx,tax=tax)
        flx = hh.ψ*lx.^hh.ζ-(-Yxtau.+1).*broadcast(*,RHS,w')
        lx = lx -sign.(flx).*dlx
    end
    
    # there are cases that FOC on labor not balanced
    # the benefit of working always surpasses the cost of working
    lx[lx.>1.0].=1.0
    lx[lx.<0.0].=0.0
    return lx
end

op_labor2 (generic function with 1 method)

In [123]:
function kt_labor(;r::Float64,w_vector::Array{Float64,1},hh::Household,RHS::Array{Float64,2},
        tax::Tax,itp_tax,Transfer::Float64,Xt::Array{Float64,2},l_prod::productivity_variables)
# first backout the mapping of (a,a')
# Use budget constraint to solve a'
# Use FOC condition to solve l
    x1,x2 = size(RHS)

    ka0 = -hh.a_max.*ones(x1,x2)
    kb0 = 1.2.*hh.a_max.*ones(x1,x2)
    
    la0 = op_labor2(r=r,w=w_vector[2*(1:l_prod.n3)],hh=hh,k_grid=ka0[:,2*(1:l_prod.n3)],RHS=RHS[:,2*(1:l_prod.n3)],tax=tax,itp_tax=itp_tax)
    lb0 = op_labor2(r=r,w=w_vector[2*(1:l_prod.n3)],hh=hh,k_grid=kb0[:,2*(1:l_prod.n3)],RHS=RHS[:,2*(1:l_prod.n3)],tax=tax,itp_tax=itp_tax)
#     lt0=(broadcast(*,RHS,hh.w*hh.z_chain.state_values')).^(1/hh.ζ)./(hh.ψ^(1/hh.ζ))
    ya_tax = Array{Float64,2}(undef,x1,x2)
    yb_tax = Array{Float64,2}(undef,x1,x2)

    ya = r.*ka0+broadcast(*,kron(la0,l_prod.p0.state_values'),w_vector')
    yb = r.*kb0+broadcast(*,kron(lb0,l_prod.p0.state_values'),w_vector')
    for j = 1:x2
        ya_tax[:,j] = itp_tax(ya[:,j])
        yb_tax[:,j] = itp_tax(yb[:,j])
    end
    ya_tax[ya_tax.<0].=0.0
    yb_tax[yb_tax.<0].=0.0
    
    Xt_ext = kron(Xt,ones(1,l_prod.n4))
    BCa = Xt_ext-(1+r).*ka0-broadcast(*,kron(la0,l_prod.p0.state_values'),w_vector')+ya_tax.-Transfer
    BCb = Xt_ext-(1+r).*kb0-broadcast(*,kron(lb0,l_prod.p0.state_values'),w_vector')+yb_tax.-Transfer
    
    if any(BCa.*BCb.>0)
        println("solution of budget constraint not bracketed")
    end
    dkx = 0.5*(kb0-ka0)
    kbx = ka0+dkx;

    tol_dkx=10^-6;
    i=0;
    lbx = Array{Float64,2}(undef,x1,x2)
    yx_tax = Array{Float64,2}(undef,x1,x2)
    
    while any(norm(dkx)>tol_dkx)
        i=i+1;
        dkx = 0.5*dkx;
        lbx=op_labor2(r=r,w=w_vector[2*(1:l_prod.n3)],hh=hh,k_grid=kbx[:,2*(1:l_prod.n3)],RHS=RHS[:,2*(1:l_prod.n3)],tax=tax,itp_tax=itp_tax);
        yx = r.*kbx+broadcast(*,kron(lbx,l_prod.p0.state_values'),w_vector')
        for j = 1:x2
            yx_tax[:,j]=itp_tax(yx[:,j])
        end
        yx_tax[yx_tax.<0].=0.0
        BCx = Xt_ext-(1+r).*kbx-broadcast(*,kron(lbx,l_prod.p0.state_values'),w_vector')+yx_tax.-Transfer
        kbx = kbx + sign.(BCx).*dkx;   
    end
    
    for j = 1:x2
        index_duplicate = findall(kbx[1:end-1,j].>=kbx[2:end,j])
        kbx[index_duplicate,j] = kbx[index_duplicate.+1,j] .-0.000001
    end
    return kbx,lbx
end

kt_labor (generic function with 1 method)

In [124]:
function Income_Asset_Info(;a_vals::Array{Float64,1},lt::Array{Float64,2},wt::Array{Float64,1},
                                r::Float64,itp_tax,tax::Tax,Transfer::Float64)
"""
    Calculate Xt,Yt,Ytax,Ytau,Ytau_r given personal info
    """  
    n = length(wt)
    Yt = repeat(a_vals,1,n).*r+broadcast(*,lt,wt')
    Ytax = Array{Float64,2}(undef,length(a_vals),n)
    for j = 1:n
        Ytax[:,j] = [itp_tax(x) for x in Yt[:,j]]
    end
    Ytax[Ytax.<0.0].=0.0
    Ytau = marginaltax(y=Yt,tax=tax)
    Ytau_r = (-Ytau.+1.0).*r.+1
    Xt = broadcast(+,Yt-Ytax.+Transfer,a_vals)
    
    return Xt,Yt,Ytax,Ytau,Ytau_r
end

Income_Asset_Info (generic function with 1 method)

In [125]:
function policy_EGM(;hh::Household,policy::policy_variables,firsttime::Int64,tax::Tax,r::Float64,w::Float64,
        Ls::Float64,l_prod::productivity_variables,error::error_recording)  
    """Partial equilibrium: Solve for policy function given prices"""
 
###########################

    tol_x = 1e-5
    tol_l = 1e-5
    diff_l = tol_l+1
    iter_l = 0
    maxiter_x=2000
    maxiter_l=100

    tol_Ls = 1e-3
    diff_Ls = tol_Ls+1
    iter_Ls = 0
    maxiter_Ls = 5
    
    k = ((r+hh.δ)/hh.α)^(1/(hh.α-1))
    tax,Transfer,itp_tax = tax_policy(Tax=tax,r=r,w=w,k=k,L=Ls)

    # prod on persistent grid
    ext_prod_n2 = kron(l_prod.pt.state_values,ones(length(hh.β_chain.state_values)))
    # prod on all grid excluding 0 earnings state
    ext_prod_n3 = kron(ext_prod_n2,l_prod.vt.state_values)
    # prod on all grid
    ext_prod_n1 = kron(ext_prod_n3,l_prod.p0.state_values)

    # discount factor on persistent grid
    ext_β = kron(ones(1,length(l_prod.pt.weights)),hh.β_chain.state_values')

    if firsttime==1
        lpt = 0.3*ones(hh.a_size,l_prod.n3)
        Xpt,~,~,~,Yptau_r=Income_Asset_Info(a_vals=hh.a_vals,lt=kron(lpt,l_prod.p0.state_values'),
                                                wt=w*ext_prod_n1,r=r,itp_tax=itp_tax,tax=tax,Transfer=Transfer)

        Xt=Xpt[:,l_prod.n4*(1:l_prod.n2)]
        Kppt = Array{Float64,2}(undef,hh.a_size,l_prod.n1)
    else
        lpt=policy.lpt_grid
        Xpt,~,~,~,Yptau_r=Income_Asset_Info(a_vals=hh.a_vals,lt=kron(lpt,l_prod.p0.state_values'),
                                                wt=w*ext_prod_n1,r=r,itp_tax=itp_tax,tax=tax,Transfer=Transfer)
        Xt=policy.Xt
        Kppt = policy.Kppt
    end

    RHS = Array{Float64,2}(undef,hh.a_size,l_prod.n2)
    #first assume labor supply fixed
    C_min = repeat(range(1,length=hh.a_size).*1e-8,1,l_prod.n1)

    while (diff_l>tol_l)&&(iter_l<maxiter_l)
# #     # for iiii=1:20
        iter_l+=1
        iter_x=0
        diff_x = tol_x+1
        while (diff_x>tol_x)&&(iter_x<maxiter_x)
            iter_x+=1
            Xt_ext  = kron(Xt,ones(1,l_prod.n4))
            for j = 1:l_prod.n1
                Kppt[:,j]=vector_linear_interpolation(Xt_ext[:,j],hh.a_vals,Xpt[:,j])
            end
            # Deal with borrowing constraint
            Kppt[Kppt.<hh.a_min].=hh.a_min
            Cpt = Xpt-Kppt
            RHS = broadcast(*,(Cpt.^(-hh.σ).*Yptau_r)*l_prod.trans_matrix',ext_β)
            Ct0 = RHS.^(-1.0/hh.σ)
            Xt1 = broadcast(+,Ct0,hh.a_vals)
            diff_x = norm(Xt1-Xt)
            Xt=Xt1
    #         println(iter_x," ",diff_x)
        end
        error.Xt = diff_x
        #update labor supply
        # first backout the mapping of (a,a') and obtain corresponding labor supply using RHS in Euler equation
        lpt1 = op_labor1(r=r,w=w.*ext_prod_n3,hh=hh,kprime=Kppt[:,l_prod.n5*(1:l_prod.n3)],
                        k_grid=repeat(hh.a_vals,1,l_prod.n3),itp_tax=itp_tax,
                        Transfer=Transfer,tax=tax)
        Xpt1,~,~,~,Yptau_r1=Income_Asset_Info(a_vals=hh.a_vals,lt=kron(lpt1,l_prod.p0.state_values'),
                                        wt=w*ext_prod_n1,r=r,itp_tax=itp_tax,tax=tax,Transfer=Transfer)
        cp1 = Xpt1-Kppt
#         cp1[cp1.<=0.0].=C_min[cp1.<=0.0]
        cp1[cp1.<=0.0].= 0.0001

        RHS = broadcast(*,(cp1.^(-hh.σ).*Yptau_r1)*l_prod.trans_matrix',ext_β)
        C0 = RHS.^(-1/hh.σ)
        #Check convergence
        diff_l = norm(lpt1-lpt)
#         println(iter_l," ",diff_l," ",norm(Yptau_r-Yptau_r1))
        
        # if lpt is fluctuating and 
        if diff_l<0.03
            dl = abs.(lpt1-lpt)
            count_l = sum(dl.>0.03)
            if count_l/(hh.a_size*l_prod.n1)<0.005
                policy.Xt=Xt
                policy.Kppt = Kppt
                policy.lpt_grid = lpt
                error.Lpt = diff_l
                break
            end
        end
        if diff_l<=tol_l || iter_l>20
            policy.Xt=Xt
            policy.Kppt = Kppt
            policy.lpt_grid = lpt
            error.Lpt = diff_l
            break
        end
        Xpt = Xpt1
        Xt = broadcast(+,C0,hh.a_vals)
        lpt = lpt1
        Yptau_r = Yptau_r1
    end
    k0,lt0 = kt_labor(;r=r,w_vector=w.*ext_prod_n1,
        hh=hh,RHS=kron(RHS,ones(1,l_prod.n4)),tax=tax,itp_tax=itp_tax,Transfer=Transfer,Xt=Xt,l_prod=l_prod)

    policy.Kt=k0
    policy.Xt=Xt
    policy.Kppt = Kppt
    policy.lpt_grid = lpt
#     ################################################################################
#     ################################################################################
#     # Error Correction
#     # 
#     # Theoratically using cash-on-hand or savings to predict savings policy is
#     # identical
#     #
#     #
#     # However, since kgrid is really large in our paper, linear approximation
#     # will lead to large differences when using cash-on-hand(Xt) and capital(kt)
#     ################################################################################
#     ################################################################################

#     diff_kpt = 1
#     it_kpt =0
#     tol_kpt = 1e-4
#     maxiter_kpt=50

#     Xt = copy(policy.Xt)
#     Kppt_l = copy(policy.Kppt)
#     while (diff_kpt>tol_kpt)&&(it_kpt<maxiter_kpt)
#         it_kpt+=1

#         k0,lt0 = kt_labor(;r=r,w=w,hh=hh,RHS=RHS,tax=tax,itp_tax=itp_tax,Transfer=Transfer,Xt=Xt)
#         Kppt = Array{Float64,2}(undef,hh.a_size,hh.z_size)
#         for j=1:hh.z_size
#             itp_kppt = LinearInterpolation(k0[:,j], hh.a_vals, extrapolation_bc = Interpolations.Linear())
#             Kppt[:,j] = itp_kppt[hh.a_vals]
#         end
#         Kppt[Kppt.<hh.a_min].=hh.a_min

#         lpt1 =op_labor1(r=r,w=w,hh=hh,kprime=Kppt,k_grid=repeat(hh.a_vals,1,hh.z_size),itp_tax=itp_tax,Transfer=Transfer)

#         Ypt1 = broadcast(+,broadcast(*,lpt1,w.*hh.z_chain.state_values'),r.*hh.a_vals)
#         Ypptax1 = itp_tax[Ypt1]
#         Ypptax1[Ypptax1.<0.0].=0.0
#         Yptau1 = marginaltax(y=Ypt1,tax=tax)

#         Xpt1 = broadcast(+,Ypt1-Ypptax1.+Transfer,hh.a_vals)
#         cp1 = Xpt1-Kppt
#         cp1[cp1.<=0.0].=C_min[cp1.<=0.0]

#         Yptau_r1 = r*(-Yptau1.+1.0).+1.0

#         RHS = hh.β.*((cp1.^(-hh.σ).*Yptau_r1)*hh.z_chain.markovchain')
#         C0 = RHS.^(-1/hh.σ)
#         Xt = broadcast(+,C0,hh.a_vals)
#         diff_kpt = norm(Kppt_l-Kppt)
#         println(it_kpt," ",diff_kpt)

#         Kppt_1 = Kppt
#         if (diff_kpt<=tol_kpt)||(it_kpt>=maxiter_kpt)
#             policy.Xt=Xt
#             policy.Kppt = Kppt
#             policy.lpt_grid = lpt1
#             policy.Kt = k0
#             break
#         end    
#     end
    
    return policy,tax,itp_tax,Transfer,error
end

policy_EGM (generic function with 1 method)

In [126]:
function cdf_asset(;r::Float64,w::Float64,hh::Household,policy::policy_variables,itp_tax,
                    Transfer::Float64,l_prod::productivity_variables,tax::Tax,Gz::Array{Float64,2})
"""Partial equilibrium: Solve for the distribution of asset given policy function"""    
    #previous asset saving as policy.Kt
    # Obtain previous asset holding for finer grid by interpolation
    a_cdf_t=Array{Float64,2}(undef,a_cdf_size,l_prod.n1)
    for j=1:l_prod.n1
        a_cdf_t[:,j] = vector_linear_interpolation(hh.a_vals, policy.Kt[:,j],hh.a_cdf_vals)
    end
    
    tol_G = 1e-4
    diff_G = 1
    iter_G = 0
    maxiter_G=2000

    while (diff_G>tol_G) && (iter_G<maxiter_G)
        iter_G+=1
        G_sz = Array{Float64,2}(undef,a_cdf_size,l_prod.n1)
        Gz_ext  = kron(Gz,ones(1,l_prod.n4))
        # Obtain previous CDF by interpolation
        for j=1:l_prod.n1
            G_sz[:,j] = vector_linear_interpolation(hh.a_cdf_vals, Gz_ext[:,j],a_cdf_t[:,j])
        end
        G_sz[a_cdf_t.<hh.a_min].=0
        G_sz[a_cdf_t.>=hh.a_max].=1

        Gz1 = G_sz*l_prod.trans_matrix'
        Gz1 = broadcast(/,Gz1,Gz1[end,:]')
        diff_G = norm(Gz-Gz1)
        Gz = Gz1
    #     println("iter_G: ",iter_G," ",diff_G)
    end
    
    # unconditional weights for persistent varible
    uncweights = kron(l_prod.pt.weights,hh.β_chain.weights)
    # obtain the aggregate capital supply
    Ks = 0.5*(hh.a_cdf_vals[1:end-1]+hh.a_cdf_vals[2:end])'*(Gz[2:end,:]-Gz[1:end-1,:])*uncweights+
    hh.a_cdf_vals[1]*Gz[1,:]'*uncweights
    # obtain the aggregate labor supply
    Kppi = Array{Float64,2}(undef,hh.a_cdf_size,l_prod.n1)
    for j = 1:l_prod.n1
        Kppi[:,j] = vector_linear_interpolation(hh.a_vals, policy.Kppt[:,j],hh.a_cdf_vals)
    end
    Kppi[Kppi.<hh.a_min].=hh.a_min

    w_vector = kron(kron(l_prod.pt.state_values,ones(length(hh.β_chain.state_values))),l_prod.vt.state_values)
    lsi = op_labor1(r=r,w=w_vector,hh=hh,kprime=Kppi[:,l_prod.n5*(1:l_prod.n3)],
        k_grid=repeat(hh.a_cdf_vals,1,l_prod.n3),itp_tax=itp_tax,Transfer=Transfer,tax=tax)
    
    Gz_ext = kron(Gz,ones(1,2))
    uncweights_vt = kron(uncweights,l_prod.vt.weights)
    Ls1 = sum(0.5*(lsi[1:end-1,:]+lsi[2:end,:]).*(Gz_ext[2:end,:]-Gz_ext[1:end-1,:])*uncweights_vt)+(lsi[1,:].*Gz_ext[1,:])'*uncweights_vt
    return Gz,Ks,Ls1,lsi
end

cdf_asset (generic function with 1 method)

In [127]:
function partial_equilibrium(;r::Float64,hh::Household,policy::policy_variables,
        firsttime::Integer,tax::Tax,aggregate::aggregate_variables,l_prod::productivity_variables,error::error_recording)
"""
    Given exogenous interest rate
    Ouput household policy function + asset supply distribution
    + implied aggregate variables
    """
    
    k = ((r+hh.δ)/hh.α)^(1/(hh.α-1))
    w = (1-hh.α)*k^hh.α

    if firsttime==1
        Ls=0.3
    else
        Ls = aggregate.L
    end

    tol_Ls = 1e-3
    diff_Ls = tol_Ls+1
    iter_Ls = 0
    maxiter_Ls = 5
    while (diff_Ls>tol_Ls)&&(iter_Ls<maxiter_Ls)
        iter_Ls+=1
        if iter_Ls>1
            firsttime = 0
        end

        # Solve policy function using Endogenous Grid Method
        policy,tax,itp_tax,Transfer,error = policy_EGM(r=r,w=w,hh=hh,policy=policy,
                                    firsttime=firsttime,l_prod=l_prod,tax=tax,Ls=Ls,error=error)
        
        #initialize CDF
        if firsttime==1
            Gz = (repeat(hh.a_cdf_vals,1,l_prod.n2).-minimum(hh.a_cdf_vals))./
                                (maximum(hh.a_cdf_vals)-minimum(hh.a_cdf_vals))
        else
            Gz = aggregate.Gz
        end
        # # Solve for distribution given policy function
        Gz,K,Ls1,lpti_grid = cdf_asset(r=r, w=w, hh=hh,policy=policy,
            itp_tax=itp_tax, Transfer=Transfer, l_prod=l_prod, tax=tax, Gz=Gz);
        
        aggregate.K = K
        aggregate.Gz = Gz
        aggregate.L = Ls1
        policy.lpti_grid = lpti_grid
        
        diff_Ls = norm(Ls1-Ls)
#         println(iter_Ls," diff in Ls: ",diff_Ls," Current Ls: ",Ls1)
        if (iter_Ls>=maxiter_Ls)||(diff_Ls<=tol_Ls)
            break
        end
        Ls = 0.3*Ls1+0.7*Ls
    end
    # obtain the implied demand side interest rate given Ks and Ls
    rs = hh.α*(aggregate.K/aggregate.L)^(hh.α-1)-hh.δ
    fr = rs-r
    println("current labor supply: ",aggregate.L)
    aggregate.r = rs
    aggregate.w = (1-hh.α)*(aggregate.K/aggregate.L)^(hh.α)
        
    return fr,policy,aggregate,error
end

partial_equilibrium (generic function with 1 method)

In [128]:
function bisection(;rmin::Float64,rmax::Float64,f::Function,hh::Household,tax::Tax,l_prod::productivity_variables)
    policy = policy_variables(Array{Float64,2}(undef,hh.a_size,l_prod.n2),
                              Array{Float64,2}(undef,hh.a_size,l_prod.n1),
                              Array{Float64,2}(undef,hh.a_size,l_prod.n1),
                              Array{Float64,2}(undef,hh.a_size,l_prod.n3),
                              Array{Float64,2}(undef,hh.a_size,l_prod.n3),
                              Array{Float64,2}(undef,hh.a_size,l_prod.n3))
    
    # Initialize Aggregate Variables
    aggregate = aggregate_variables(0.0,0.0,0.0,0.0,0.0,
            tax,Array{Float64,2}(undef,hh.a_size,l_prod.n2))

    error = error_recording(0.0,0.0,0.0,0.0,0.0,0.0,0.0)
    fa = 1
    fb = -1
    a = rmin
    b = rmax
#     fa,~,~= f(r=a,hh=hh,policy=policy,firsttime=1,tax=tax,aggregate=aggregate,l_prod=l_prod,error=error)
#     fb,~,~= f(r=b,hh=hh,policy=policy,firsttime=1,tax=tax,aggregate=aggregate,l_prod=l_prod,error=error)
    if fa.*fb>0
        println("equilibrium interest rate not braceketed!!!")
    end
    println("[",fa,",",fb,"]")
    sa = sign(fa)
    x = (a+b)*0.5
    dx = (b-a)/2
    fx = undef
    tol =1e-5
    iter = 0
    maxiter=50
    
    while any(abs(dx)>tol) && (iter<maxiter)
        iter+=1
#         fx,Gz,Ks,rs,hh = f(r=x,hh=hh)
        if iter==1
            firsttime=1
        else
            firsttime=0
        end
        fx,policy,aggregate,error = f(r=x,hh=hh,policy=policy,firsttime=firsttime,tax=tax,aggregate=aggregate,
                                 l_prod=l_prod,error=error)
        
        if sign(fx)==sign(fa)
            a = x
            x = (x+b)*0.5
            dx = dx*0.5
        else
            b = x
            x = (a+x)*0.5
            dx = dx*0.5
        end
        println(iter," Current interst rate is: [",a," , ",b,"]")
        println("gap of rs-rd=:",fx)
    end
    return fx,policy,aggregate,error
end

bisection (generic function with 1 method)

In [129]:
function income_wealth_share(;x_grid::Array{Float64,1},x_dist::Array{Float64,1},X::Float64,
                            share::Float64,top::Integer=1,positive::Integer=1)
    # Top = 1 top wealth share
    # Top = 0 Bottom wealth share
    # positive=0 meaning solving for population shares of negative wealth
    # wealth grid, distribution of wealth, aggregate wealth level
    if positive==1
        if top==1
            position = 1-share
            I_share=findnext(x_dist.>position,1)
            X_share = (0.5*(x_grid[I_share]+x_grid[I_share-1])*(x_dist[I_share]-position)+
                (x_dist[I_share+1:end,1]-x_dist[I_share:end-1,1])'*(0.5.*(x_grid[I_share+1:end]+x_grid[I_share:end-1])))/X
        else
            I_share=findnext(x_dist.>share,1)
            if I_share>1
                X_share = (x_grid[1]*x_dist[1,1]+(0.5*(x_grid[I_share]+x_grid[I_share-1]))*(0.5-x_dist[I_share-1])+
                    (x_dist[2:I_share-1,1]-x_dist[1:I_share-2,1])'*(0.5*(x_grid[1:I_share-2]+x_grid[2:I_share-1])))/X
            else
                X_share = (x_grid*x_dist[1,1])/X
            end
        end
        return X_share
    else
        In=findnext(x_grid.>0,1)
        if In==1
            return P_share=0
        else
            P_share = x_dist[In-1]+(x_dist[In]-x_dist[In-1])*(-x_grid[In-1])/(x_grid[In]-x_grid[In-1])
            return P_share
        end
    end
end

income_wealth_share (generic function with 1 method)

In [130]:
function wealth_inequality(;hh::Household,l_prod::productivity_variables,K::Float64,Gz::Array{Float64,2},
                        all::Integer=1,top::Integer=1,share=0.1,positive=1)
    uncweights=kron(l_prod.pt.weights,hh.β_chain.weights)
    Gk=Gz*uncweights
    
    if all==1
        Ksh50 = income_wealth_share(;x_grid = hh.a_cdf_vals,x_dist=Gk,X=K,share=0.5,top=0)
        Ksh10 = income_wealth_share(;x_grid = hh.a_cdf_vals,x_dist=Gk,X=K,share=0.1,top=1)
        Ksh1 = income_wealth_share(;x_grid = hh.a_cdf_vals,x_dist=Gk,X=K,share=0.01,top=1)
        Ksh01 = income_wealth_share(;x_grid = hh.a_cdf_vals,x_dist=Gk,X=K,share=0.001,top=1)
        Ksh001 = income_wealth_share(;x_grid = hh.a_cdf_vals,x_dist=Gk,X=K,share=0.0001,top=1)
        Ng_k = income_wealth_share(;x_grid = hh.a_cdf_vals,x_dist=Gk,X=K,share=0.5,positive=0)

        return Ksh50,Ksh10,Ksh1,Ksh01,Ksh001,Ng_k
    else
        K_share = income_wealth_share(;x_grid = hh.a_cdf_vals,x_dist=Gk,X=K,share=share,top=top,positive=positive)
        return K_share
    end
end

wealth_inequality (generic function with 1 method)

In [131]:
function income_inequality(;hh,l_prod,r,w,K,Lpti,k_dist)
    # note that the states for all wages include transitory labor income shocks
    uncweights=kron(kron(l_prod.pt.weights,hh.β_chain.weights),l_prod.vt.weights)
    ext_prod = kron(kron(l_prod.pt.state_values,ones(length(hh.β_chain.state_values))),l_prod.vt.state_values)
    Gz_ext = kron(k_dist,ones(1,length(l_prod.vt.state_values)))

    Y_matrix = broadcast(+,broadcast(*,Lpti,w*ext_prod'),r*hh.a_cdf_vals)
    Y_length = length(Y_matrix)

    GY_matrix = broadcast(*,uncweights',[Gz_ext[1,:]';Gz_ext[2:end,:]-Gz_ext[1:end-1,:]])
    Y_frac = Y_matrix.*GY_matrix

    Y_arrow = reshape(Y_matrix,Y_length)
    Y_frac_arrow = reshape(Y_frac,Y_length)
    GY_arrow = reshape(GY_matrix,Y_length)

    Iy = sortperm(Y_arrow)
    Y_arrow = sort(Y_arrow)# check!
    Y_frac_arrow = Y_frac_arrow[Iy]
    GY_arrow = GY_arrow[Iy]

    Ytotal = sum(Y_frac_arrow)
    CDF_Y = zeros(Y_length)
    CDF_Y[1] = GY_arrow[1]
    for j = 2:Y_length
        CDF_Y[j]= CDF_Y[j-1]+GY_arrow[j]
    end

    Ysh50 = income_wealth_share(;x_grid=Y_arrow,x_dist = CDF_Y,X=Ytotal,share=0.5,top=0)
    Ysh10 = income_wealth_share(;x_grid=Y_arrow,x_dist = CDF_Y,X=Ytotal,share=0.1)
    Ysh1 = income_wealth_share(;x_grid=Y_arrow,x_dist = CDF_Y,X=Ytotal,share=0.01)
    Ysh01 = income_wealth_share(;x_grid=Y_arrow,x_dist = CDF_Y,X=Ytotal,share=0.001)
    Ysh001 = income_wealth_share(;x_grid=Y_arrow,x_dist = CDF_Y,X=Ytotal,share=0.0001)
  
#     #log labor income volatility
#     logYs = 0
#     ysi = log.(broadcast(,ext_prod,Lpti))
#     for i = 1:l_prod.n2
#         logYs+=uncweights[i,1].*(ysi[1,i].*Gz[1,i]+(Gz[2:end,i]-Gz[1:end-1,i])'*(0.5.*(ysi[2:end,i]+ysi[1:end-1,i])))
#     end
#     logvarY=0;
#     for i=1:l_prod.n2
#         logvarY=logvarY+uncweights[i,1].*((ysi[1,i]-logYs)^2.*Gz[1,i]+(Gz[2:end,i]-Gz[1:end-1,i])'*((0.5.*(ysi[2:end,i]+ysi[1:end-1,i])-logYs).^2));
#     end

#     stdlogvarY(iii,1)=sqrt(logvarY);  
    return Ysh50,Ysh10,Ysh1,Ysh01,Ysh001
end

income_inequality (generic function with 1 method)

In [132]:
function steady_state(;hh::Household,tax::Tax,l_prod::productivity_variables)
    #(Xt,Kt,Kppt,lt_egm,lt_grid,lpt_grid)
    policy = policy_variables(Array{Float64,2}(undef,hh.a_size,l_prod.n2),
                              Array{Float64,2}(undef,hh.a_size,l_prod.n1),
                              Array{Float64,2}(undef,hh.a_size,l_prod.n1),
                              Array{Float64,2}(undef,hh.a_size,l_prod.n3),
                              Array{Float64,2}(undef,hh.a_size,l_prod.n3),
                              Array{Float64,2}(undef,hh.a_size,l_prod.n3))

    aggregate = aggregate_variables(0.0,0.0,0.0,0.0,0.0,
            tax,Array{Float64,2}(undef,hh.a_size,l_prod.n2))

    error_compute = error_recording(0.0,0.0,0.0,0.0,0.0,0.0,0.0)
    
    @time fr,policy,aggregate,error = bisection(rmin=0.001,rmax=0.08,
                            f=partial_equilibrium,hh=hh,tax=tax,l_prod=l_prod);
    return fr,policy,aggregate,error
#     return fr
end

steady_state (generic function with 1 method)

# Main File

## Initialization

## Computing Steady State equilibrium for 1967 and long run

In [133]:
using LinearAlgebra
using Interpolations
import FastGaussQuadrature.gausshermite
using Distributions
using SparseArrays
using JLD2
using SharedArrays
using Distributed

# Firm parameter
α = 0.36 #capital share
δ = 0.048 #depreciation rate

# Consumer parameter
σ = 1.5
β_μ = 0.92
ψ = 27.0
ζ = 2.0

# government parameter
λT = 0.6 #fraction of revenue distribution as transfer

# stochastic discounting
β_markovchain,β_state_values,β_weight = GaussHermite_matrix(n=15,rho=0.992,
                                        σ=sqrt(0.0021^2/(1-0.992^2)),μ=β_μ,exponential=0)
β_chain = MarkovChain(β_markovchain,β_state_values,β_weight)

##############
a_max = 1e6
# a_min = 1e-10
a_min = -0.24
a_size = 50
a_cdf_size = 500

# Generate an instance of households with grids, utility setup
hh = parameters(σ=σ,β_chain=β_chain,α=α,δ=δ,a_min=a_min,a_max=a_max,
            ψ=ψ,ζ=ζ, a_size=a_size,a_cdf_size=a_cdf_size,λT=λT)

##################################################################
# Load Income Data
# @load "/Users/mac/Dropbox/Julia/Aiyagari_EGM/income_simulation.jld2" income_prod_list
@load "C:\\Users\\leosh\\Dropbox\\Julia\\Aiyagari_EGM\\income_simulation.jld2" income_prod_list

labor_productivity = []

shrink = 1
for i =1:2
    if i==1
        ###############
        # Persistent productivity
        ρ_pt = income_prod_list.ρ
        σ_pt = income_prod_list.σ_pt[1]
        kt = income_prod_list.kt[1]
        ###############
        # Transitory productivity
        σ_vt = income_prod_list.σ_vt[1]
    else
        ###############
        # Persistent productivity
        ρ_pt = income_prod_list.ρ
        # Here last period σ_pt is implied from σ_pt_error
        σ_pt = sqrt(income_prod_list.σ_pt_error[end]^2/(1-ρ_pt^2))
#         σ_pt = income_prod_list.σ_pt[end]
        kt = income_prod_list.kt[end]
        ###############
        # Transitory productivity
        σ_vt = income_prod_list.σ_vt[end]        
    end
    # Number of grid for pt is pre-defined as 17, specified at each population percentile
    n = 15 # number of Gausss-Hermite nodes 
    pt_chain = ptmatrix(n=15, ρ_pt=ρ_pt,σ_pt=σ_pt,kt=kt,shrink=1)
    
    Nv = 2
    v_nodes,v_weight = gausshermite(Nv)
    v_weight = v_weight./sqrt(pi)
    v_grid = sqrt(2)*σ_vt*v_nodes
    v_grid = exp.(v_grid)./exp(σ_vt^2/2)
    vt_chain = MarkovChain(repeat(v_weight',length(v_grid),1),v_grid,v_weight)
    ###############
    # zero earning state
    p0 = MarkovChain([0.075 0.925;0.075 0.925],[0,1],[0.075,0.925])
    ##############
    # group all productivity into one struct and a related transitional matrix
    labor_product = productivity(pt=pt_chain,vt=vt_chain,p0=p0,β_chain=β_chain)
    
    append!(labor_productivity,[labor_product])
end

##################################################################
# Load Tax Data
# @load "/Users/mac/Dropbox/Julia/Aiyagari_EGM/tax_system.jld2" tax_policy_history
@load "C:\\Users\\leosh\\Dropbox\\Julia\\Aiyagari_EGM\\tax_system.jld2" tax_policy_history


tax_policy_variables = []

for i = 1:2
# Initialize tax system
    if i ==1
        y_pshare = tax_policy_history.y_pshare
        y_ishare = tax_policy_history.y_ishare[:,1]
        y_atax=tax_policy_history.y_atax[:,1]
    else
        y_pshare = tax_policy_history.y_pshare
        y_ishare = tax_policy_history.y_ishare[:,end]
        y_atax=tax_policy_history.y_atax[:,end]
    end
    #(yp_share,yi_share,y_atax,y_mtax,y_cutoff,y_end_tax)
    tax = Tax(y_pshare,y_ishare,y_atax,Array{Float64,1}(undef,11),Array{Float64,1}(undef,11),
            Array{Float64,1}(undef,11))  
    append!(tax_policy_variables,[tax])
end
##################################################################
# Initializing steady state results for first period and final period
fx_v = Array{Float64,1}(undef,2)
aggregate_v = Vector(undef, 2)
policy_v = Vector(undef, 2)
diff_v = Vector(undef, 2)


SystemError: SystemError: opening file C:\Users\leosh\Dropbox\Julia\Aiyagari_EGM\income_simulation.jld2: No such file or directory

In [32]:
for i = 1:2
    fx_v[i],policy_v[i],aggregate_v[i],diff_v[i] = steady_state(hh=hh,
                                        l_prod=labor_productivity[i],tax=tax_policy_variables[i])   
end

[1,-1]
current labor supply: 0.3357427907341346
1 Current interst rate is: [0.0405 , 0.08]
gap of rs-rd=:0.0069134296252313585
current labor supply: 0.30912267868406634
2 Current interst rate is: [0.0405 , 0.06025]
gap of rs-rd=:-0.04731303566814914
current labor supply: 0.3241539188590581
3 Current interst rate is: [0.0405 , 0.050375]
gap of rs-rd=:-0.020808076770753664
current labor supply: 0.3291607039466145
4 Current interst rate is: [0.0405 , 0.045437500000000006]
gap of rs-rd=:-0.009697879562881193
current labor supply: 0.3320259352019708
5 Current interst rate is: [0.0405 , 0.04296875]
gap of rs-rd=:-0.002614996288649682
current labor supply: 0.33361922741799066
6 Current interst rate is: [0.041734375000000004 , 0.04296875]
gap of rs-rd=:0.001990207939769048
current labor supply: 0.3327235830895926
7 Current interst rate is: [0.041734375000000004 , 0.0423515625]
gap of rs-rd=:-0.00010820387915119051
current labor supply: 0.3331888454174466
8 Current interst rate is: [0.042042968

In [34]:
using JLD2
# @save "/Users/mac/Dropbox/Julia/Aiyagari_EGM/steady_state.jld2" fx_v policy_v aggregate_v diff_v hh labor_productivity tax_policy_variables

@save "C:\\Users\\leosh\\Dropbox\\Julia\\Aiyagari_EGM\\steady_state.jld2" fx_v policy_v aggregate_v diff_v hh labor_productivity tax_policy_variables


In [134]:
@load "/Users/mac/Dropbox/Julia/Aiyagari_EGM/steady_state.jld2" fx_v policy_v aggregate_v diff_v hh labor_productivity tax_policy_variables

# @load "C:\\Users\\leosh\\Dropbox\\Julia\\Aiyagari_EGM\\steady_state.jld2" fx_v policy_v aggregate_v diff_v hh labor_productivity tax_policy_variables

7-element Array{Symbol,1}:
 :fx_v                
 :policy_v            
 :aggregate_v         
 :diff_v              
 :hh                  
 :labor_productivity  
 :tax_policy_variables

### obtaining the inequality measure for both capital and income

In [135]:
using Plots
using Formatting: printfmt
Ksh50 = Array{Float64,1}(undef,2); Ksh10 = Array{Float64,1}(undef,2); 
Ksh1 = Array{Float64,1}(undef,2); Ksh01 = Array{Float64,1}(undef,2);
Ksh001=Array{Float64,1}(undef,2); Ng_k=Array{Float64,1}(undef,2); 
Ysh50 = Array{Float64,1}(undef,2); Ysh10 = Array{Float64,1}(undef,2);
Ysh1 = Array{Float64,1}(undef,2); Ysh01 = Array{Float64,1}(undef,2);
Ysh001=Array{Float64,1}(undef,2);

for i=1:2
    Ksh50[i],Ksh10[i],Ksh1[i],Ksh01[i],Ksh001[i],Ng_k[i] = wealth_inequality(;hh=hh,l_prod=labor_productivity[i],K=aggregate_v[i].K,Gz=aggregate_v[i].Gz)
    Ysh50[i],Ysh10[i],Ysh1[i],Ysh01[i],Ysh001[i]= income_inequality(hh=hh,l_prod=labor_productivity[i],r=aggregate_v[i].r,
                                                w=aggregate_v[i].w,K=aggregate_v[i].K,Lpti=policy_v[i].lpti_grid,k_dist=aggregate_v[i].Gz)
end

printfmt("Wealth share in 1967 for bottom 50% is {:.2f}, top 10% is {:.2f}, top 1% is {:.2f}, top 0.1% is {:.2f}, top 0.01% is {:.2f} \n",Ksh50[1],Ksh10[1],Ksh1[1],Ksh01[1],Ksh001[1])
printfmt("Wealth share in LR   for bottom 50% is {:.2f}, top 10% is {:.2f}, top 1% is {:.2f}, top 0.1% is {:.2f}, top 0.01% is {:.2f} \n",Ksh50[2],Ksh10[2],Ksh1[2],Ksh01[2],Ksh001[2])

printfmt("Income share in 1967 for bottom 50% is {:.2f}, top 10% is {:.2f}, top 1% is {:.2f}, top 0.1% is {:.2f}, top 0.01% is {:.3f} \n",Ysh50[1],Ysh10[1],Ysh1[1],Ysh01[1],Ysh001[1])
printfmt("Income share in LR   for bottom 50% is {:.2f}, top 10% is {:.2f}, top 1% is {:.2f}, top 0.1% is {:.2f}, top 0.01% is {:.3f} \n",Ysh50[2],Ysh10[2],Ysh1[2],Ysh01[2],Ysh001[2])


Wealth share in 1967 for bottom 50% is 0.07, top 10% is 0.66, top 1% is 0.27, top 0.1% is 0.07, top 0.01% is 0.01 
Wealth share in LR   for bottom 50% is 0.01, top 10% is 0.88, top 1% is 0.62, top 0.1% is 0.30, top 0.01% is 0.10 
Income share in 1967 for bottom 50% is 0.28, top 10% is 0.27, top 1% is 0.07, top 0.1% is 0.02, top 0.01% is 0.004 
Income share in LR   for bottom 50% is 0.18, top 10% is 0.42, top 1% is 0.17, top 0.1% is 0.06, top 0.01% is 0.020 


## Function related to transtional dynamics

In [136]:
mutable struct Tax_path
    """
    Record the tax policy along transitional policy
    """
    y_pshare::Array{Float64,1}
    y_ishare::Array{Float64,2}
    y_mtax::Array{Float64,2}
    y_atax::Array{Float64,2}   
end

In [137]:
mutable struct MarkovChain_dynamics
    """
    records the persistent labor productivity dynamics along the transitional path
    """
    markovchain_forward::Array{Float64,2}
    markovchain_backward::Array{Float64,2}
    state_values::Array{Float64,1}
    weights::Array{Float64,1}
end

In [138]:
mutable struct productivity_path
    """
    record all labor productivity dynamics along the transitional path
    Related numbers
    """
    pt_path::Array{MarkovChain_dynamics,1}
    vt_path::Array{MarkovChain,1}
    p0::MarkovChain
    n1::Integer
    n2::Integer
    n3::Integer
    n4::Integer
    n5::Integer
end

In [139]:
function tax_path_construction(tax_policy_history,T::Integer)
    """
    Construct the tax policy along the transitional path
    """
    n_tax_state, T_history = size(tax_policy_history.y_atax)
    #Initialization
    tax_path = Tax_path(tax_policy_history.y_pshare,Array{Float64,2}(undef,n_tax_state,T),
                Array{Float64,2}(undef,n_tax_state,T),Array{Float64,2}(undef,n_tax_state,T))
    tax_path.y_ishare[:,1:T_history] = tax_policy_history.y_ishare
    tax_path.y_mtax[:,1:T_history] = tax_policy_history.y_mtax
    tax_path.y_atax[:,1:T_history] = tax_policy_history.y_atax
    
    #After 2010, tax system fixed(relative to income share)
    for i=T_history+1:T
        tax_path.y_ishare[:,i] .= tax_policy_history.y_ishare[:,end]
        tax_path.y_mtax[:,i] .= tax_policy_history.y_mtax[:,end]
        tax_path.y_atax[:,i] .= tax_policy_history.y_atax[:,end]
    end
    return tax_path
end

tax_path_construction (generic function with 1 method)

In [140]:
function ptmatrix_path_construction(income_prod_list,T::Integer,shrink::Float64=1.0,n::Integer=15)
    perc_pt=[0.0001, 0.01, 0.1, 0.25, 0.5, 0.75, 0.9,
        0.925, 0.95, 0.975, 0.99, 0.999, 0.9999, 0.99999, 0.999999, 0.9999999, 0.99999999]
    m = length(perc_pt)
    n_history = length(income_prod_list.kt)
    
    ρ = income_prod_list.ρ
    trans_pt = zeros(m,T)
    trans_φ = zeros(m,T)
    λ_matrix_forward_T = Vector(undef,T-1)
    λ_matrix_backward_T = Vector(undef,T-1)
    
    kt_T = zeros(T)
    kt_T[1:n_history] = income_prod_list.kt
    kt_T[n_history+1:T] .= income_prod_list.kt[end]
    
    σ_error_T = zeros(T,1)
    σ_error_T[1:n_history] = income_prod_list.σ_pt_error
    σ_error_T[n_history+1:end] .= income_prod_list.σ_pt_error[end]
        
    σ_pt_T = zeros(T)
    σ_pt_T[1] = income_prod_list.σ_pt[1]
    σ_pt_implied = sqrt(σ_error_T[end]^2/(1-ρ^2))
    for i = 2:T
        σ_pt_T[i] = sqrt(ρ^2*σ_pt_T[i-1]^2+σ_error_T[i]^2)
        if norm(σ_pt_T[i]-σ_pt_implied)<1e-5
            σ_pt_T[i] = σ_pt_implied
        end
    end
    
    λp_NN_matrix_T = Vector(undef,T)
    #########################################################################################################
    # Productivity grid, unconditional distribution
    #########################################################################################################    
    for i = 1:T
        σ_pt = σ_pt_T[i]
        kt = kt_T[i]
        
        pt = quantile.(Normal(0,σ_pt),perc_pt)
        xm = pt[7] # the threshold F(pt)=0.9
        φ_ptail = quantile.(Pareto(kt,xm),(perc_pt[8:end].-0.9)./(1-0.9))
        φ = exp.(pt)
        scale_ptail = φ[7]/quantile.(Pareto(kt,xm),0)
        φ[8:end] = φ_ptail*scale_ptail
        φ = exp.(log.(φ)./shrink)
        ###############################################################
        # Obtain the unconditional distribution
        λp_NN_matrix = pt_unconditional_distribution(pt=pt,φ=φ,σ_pt=σ_pt)
        φ = φ./(λp_NN_matrix'*φ)
        trans_pt[:,i] = pt
        trans_φ[:,i] = φ
        λp_NN_matrix_T[i] = λp_NN_matrix
    end
    
    p_nodes,p_weight = gausshermite(n) 
    p_weight = p_weight./sum(p_weight)
    
    for i = 1:T-1
    #########################################################################################################
    # Forward Matrix
    #########################################################################################################
        pn_matrix_forward = zeros(m, n)
        φ_n_matrix = zeros(m, n)
        for j = 1:n
            pn_matrix_forward[:,j] = ρ*trans_pt[:,i].+sqrt(2)*σ_error_T[i+1]*p_nodes[j]
            ipt_φ_matrix = LinearInterpolation(trans_pt[:,i+1], trans_φ[:,i+1], extrapolation_bc = Line())
            φ_n_matrix[:,j] = ipt_φ_matrix([p for p in pn_matrix_forward[:,j]])            
        end
        
        λ_matrix_forward = integration_matrix(n_matrix = φ_n_matrix,n_grid = trans_φ[:,i+1],weight = p_weight)
        λ_matrix_forward_T[i] = λ_matrix_forward
    #########################################################################################################
    # Backward Matrix
    #########################################################################################################        
        pn_matrix_backward = zeros(m, n)
        φ_n_matrix = zeros(m, n)
        for j = 1:n
            pn_matrix_backward[:,j] = ρ*trans_pt[:,i+1].+sqrt(2)*σ_error_T[i]*p_nodes[j]
            ipt_φ_matrix = LinearInterpolation(trans_pt[:,i], trans_φ[:,i], extrapolation_bc = Line())
            φ_n_matrix[:,j] = ipt_φ_matrix([p for p in pn_matrix_backward[:,j]])            
        end
        
        λ_matrix_backward = integration_matrix(n_matrix = φ_n_matrix,n_grid = trans_φ[:,i],weight = p_weight)
        λ_matrix_backward_T[i] = λ_matrix_backward        
    end
    
    pt_path = Vector(undef,T)
    for i = 1:T-1
        pt_path[i] = MarkovChain_dynamics(λ_matrix_forward_T[i],λ_matrix_backward_T[i],trans_φ[:,i],λp_NN_matrix_T[i])
    end
    pt_path[T] = MarkovChain_dynamics(λ_matrix_forward_T[end],λ_matrix_backward_T[end],trans_φ[:,end],λp_NN_matrix_T[end])
    return pt_path
end

ptmatrix_path_construction (generic function with 3 methods)

In [141]:
function vt_path_construction(σ_vt_history::Array{Float64,1},T::Integer,shrink::Float64=1)
    vt_path = Vector(undef,T)
    N_vt = length(σ_vt_history)
    Nv = 2
    v_nodes,v_weight = gausshermite(Nv)
    v_weight = v_weight./sqrt(pi)
    for i = 1:T
        if i<=N_vt
            σ_vt = σ_vt_history[i]
        else
            σ_vt = σ_vt_history[end]
        end
        v_grid = sqrt(2)*σ_vt*v_nodes
        v_grid = exp.(v_grid)./exp(σ_vt^2/2)
        v_grid = exp.(log.(v_grid)./shrink)
        vt_chain = MarkovChain(repeat(v_weight',length(v_grid),1),v_grid,v_weight) 
        vt_path[i] = vt_chain
    end
    return vt_path
end

vt_path_construction (generic function with 2 methods)

In [142]:
function op_labor2_ktlabor(;r::Float64,w,hh::Household,
        k_grid,tax::Tax,RHS,itp_tax)
"""
    Solve labor optimality condition using RHS of Euler equation for t and t+1
    be careful of the period t of RHS when using.
    """
    x1,x2 = size(k_grid)
    
    
    
    la = zeros(x1*x2).+0.001
    lb = ones(x1*x2)
    # labor disutility - benefit of labor supplying
    
    Ya = k_grid.*r+broadcast(*,la,w')
    Yatau = marginaltax(y=Ya,tax=tax)
    
    Yb = k_grid.*r+broadcast(*,lb,w')
    Ybtau = marginaltax(y=Yb,tax=tax)
    
    fla = hh.ψ*la.^hh.ζ-(-Yatau.+1).*broadcast(*,RHS,w')
    flb = hh.ψ*lb.^hh.ζ-(-Ybtau.+1).*broadcast(*,RHS,w')
    
    dlx = 0.5*(lb-la)
    lx = la+dlx
    tol=1e-4
    iter = 0
    C_min = repeat(range(1,length=x1).*1e-6,1,x2)
    while any(abs.(dlx).>tol)
        iter+=1
        dlx = 0.5*dlx
        Yx = k_grid.*r+broadcast(*,lx,w')
        Yxtau = marginaltax(y=Yx,tax=tax)
        flx = hh.ψ*lx.^hh.ζ-(-Yxtau.+1).*broadcast(*,RHS,w')
        lx = lx -sign.(flx).*dlx
    end
    
    # there are cases that FOC on labor not balanced
    # the benefit of working always surpasses the cost of working
    lx[lx.>1.0].=1.0
    lx[lx.<0.0].=0.0
    return lx
end

op_labor2_ktlabor (generic function with 1 method)

In [143]:
function kt_labor_fast1(;r::Float64,w_vector::Array{Float64,1},hh::Household,RHS::Array{Float64,2},
        tax::Tax,itp_tax,Transfer::Float64,Xt::Array{Float64,2},l_prod::productivity_variables)
# first backout the mapping of (a,a')
# Use budget constraint to solve a'
# Use FOC condition to solve l
    x1,x2 = size(RHS)

    ka0 = -hh.a_max.*ones(x1,x2)
    kb0 = 1.2.*hh.a_max.*ones(x1,x2)
    
    la0 = op_labor2(r=r,w=w_vector[2*(1:l_prod.n3)],hh=hh,k_grid=ka0[:,2*(1:l_prod.n3)],RHS=RHS[:,2*(1:l_prod.n3)],tax=tax,itp_tax=itp_tax)
    lb0 = op_labor2(r=r,w=w_vector[2*(1:l_prod.n3)],hh=hh,k_grid=kb0[:,2*(1:l_prod.n3)],RHS=RHS[:,2*(1:l_prod.n3)],tax=tax,itp_tax=itp_tax)
#     lt0=(broadcast(*,RHS,hh.w*hh.z_chain.state_values')).^(1/hh.ζ)./(hh.ψ^(1/hh.ζ))
    ya_tax = Array{Float64,2}(undef,x1,x2)
    yb_tax = Array{Float64,2}(undef,x1,x2)

    ya = r.*ka0+broadcast(*,kron(la0,l_prod.p0.state_values'),w_vector')
    yb = r.*kb0+broadcast(*,kron(lb0,l_prod.p0.state_values'),w_vector')
    for j = 1:x2
        ya_tax[:,j] = itp_tax(ya[:,j])
        yb_tax[:,j] = itp_tax(yb[:,j])
    end
    ya_tax[ya_tax.<0].=0.0
    yb_tax[yb_tax.<0].=0.0
    
    Xt_ext = kron(Xt,ones(1,l_prod.n4))
    BCa = Xt_ext-(1+r).*ka0-broadcast(*,kron(la0,l_prod.p0.state_values'),w_vector')+ya_tax.-Transfer
    BCb = Xt_ext-(1+r).*kb0-broadcast(*,kron(lb0,l_prod.p0.state_values'),w_vector')+yb_tax.-Transfer
    
    if any(BCa.*BCb.>0)
        println("solution of budget constraint not bracketed")
    end
    dkx = 0.5*(kb0-ka0)
    kbx = ka0+dkx;

    tol_dkx=10^-6;
    i=0;
    lbx = Array{Float64,2}(undef,x1,x2)
    yx_tax = Array{Float64,2}(undef,x1,x2)
    
    tol_BC = 1e-5
    kbx_loop = kbx
    indexXt_loop = reshape(collect(1:x1*x2),x1,x2)
    indi_E0 = kron(ones(hh.a_size,l_prod.n1),l_prod.p0.state_values')
    RHS_ext = RHS
    Xt_loop = Xt_ext
    wage_loop = repeat(w_vector',hh.a_size,1)
    
    while any(norm(dkx)>tol_dkx)
        i=i+1;
        dkx = 0.5*dkx;
        lbx_loop=op_labor2(r=r,w=wage_loop[indi_E0.>0],hh=hh,k_grid=kbx[indi_E0.>0],RHS=RHS_ext[indi_E0.>0],tax=tax,itp_tax=itp_tax);
        lbx_bc = zeros(size(kbx_loop))
        lbx_bc[indi_E0.>0] = lbx_loop
        
        yx = r.*kbx_loop+lbx_bc.*wage_loop
        for j = 1:x2
            yx_tax[:,j]=itp_tax(yx[:,j])
        end
        yx_tax[yx_tax.<0].=0.0
        BCx = Xt_loop-(1+r).*kbx_loop-wage_loop.*lbx_bc+yx_tax.-Transfer
        kbx_loop = kbx_loop + sign.(BCx).*dkx;
        if any(any(abs.(BCx)<tol_BC))>0
            kbx[indexXt_loop[abs.(BCx)<=tol_BCx]] = kx_loop[abs.(BCx)<=tol_BCx]
            kbx_loop = kbx_loop[abs.(BCx).>tol_BCx]
            dkx = dkx[abs.(BCx).>tol_BCx]
            wage_loop = wage_loop[abs.(BCx).>tol_BCx]
            RHS_loop = RHS_loop[abs.(BCx).>tol_BCx]
            Xt_loop = Xt_loop[abs.(BCx).>tol_BCx]
            
            indexXt_loop = indexXt_loop[abs.(BCx).>tol_BCx]
            indi_E0 = indi_E0[abs.(BCx).>tol_BCx]
        end
    end
    kbx[indexXt_loop] = kbx_loop
    return kbx
end

kt_labor_fast1 (generic function with 1 method)

In [144]:
function kt_labor_fast(;r::Float64,w_vector::Array{Float64,1},hh::Household,RHS::Array{Float64,2},
        tax::Tax,itp_tax,Transfer::Float64,Xt::Array{Float64,2},l_prod::productivity_variables)
# first backout the mapping of (a,a')
# Use budget constraint to solve a'
# Use FOC condition to solve l
    x1,x2 = size(RHS)

    ka0 = -hh.a_max.*ones(x1,x2)
    kb0 = 1.2.*hh.a_max.*ones(x1,x2)
    
    la0 = op_labor2(r=r,w=w_vector[2*(1:l_prod.n3)],hh=hh,k_grid=ka0[:,2*(1:l_prod.n3)],RHS=RHS[:,2*(1:l_prod.n3)],tax=tax,itp_tax=itp_tax)
    lb0 = op_labor2(r=r,w=w_vector[2*(1:l_prod.n3)],hh=hh,k_grid=kb0[:,2*(1:l_prod.n3)],RHS=RHS[:,2*(1:l_prod.n3)],tax=tax,itp_tax=itp_tax)
#     lt0=(broadcast(*,RHS,hh.w*hh.z_chain.state_values')).^(1/hh.ζ)./(hh.ψ^(1/hh.ζ))
    ya_tax = Array{Float64,2}(undef,x1,x2)
    yb_tax = Array{Float64,2}(undef,x1,x2)

    ya = r.*ka0+broadcast(*,kron(la0,l_prod.p0.state_values'),w_vector')
    yb = r.*kb0+broadcast(*,kron(lb0,l_prod.p0.state_values'),w_vector')
    for j = 1:x2
        ya_tax[:,j] = itp_tax(ya[:,j])
        yb_tax[:,j] = itp_tax(yb[:,j])
    end
    ya_tax[ya_tax.<0].=0.0
    yb_tax[yb_tax.<0].=0.0
    
    Xt_ext = kron(Xt,ones(1,l_prod.n4))
    BCa = Xt_ext-(1+r).*ka0-broadcast(*,kron(la0,l_prod.p0.state_values'),w_vector')+ya_tax.-Transfer
    BCb = Xt_ext-(1+r).*kb0-broadcast(*,kron(lb0,l_prod.p0.state_values'),w_vector')+yb_tax.-Transfer
    
    if any(BCa.*BCb.>0)
        println("solution of budget constraint not bracketed")
    end
    dkx = 0.5*(kb0-ka0)
    kbx = ka0+dkx;

    tol_dkx=10^-6;
    i=0;
    lbx = Array{Float64,2}(undef,x1,x2)
    yx_tax = Array{Float64,2}(undef,x1,x2)
    
    while any(norm(dkx)>tol_dkx)
        i=i+1;
        dkx = 0.5*dkx;
        lbx=op_labor2(r=r,w=w_vector[2*(1:l_prod.n3)],hh=hh,k_grid=kbx[:,2*(1:l_prod.n3)],RHS=RHS[:,2*(1:l_prod.n3)],tax=tax,itp_tax=itp_tax);
        yx = r.*kbx+broadcast(*,kron(lbx,l_prod.p0.state_values'),w_vector')
        for j = 1:x2
            yx_tax[:,j]=itp_tax(yx[:,j])
        end
        yx_tax[yx_tax.<0].=0.0
        BCx = Xt_ext-(1+r).*kbx-broadcast(*,kron(lbx,l_prod.p0.state_values'),w_vector')+yx_tax.-Transfer
        kbx = kbx + sign.(BCx).*dkx;   
    end
    return kbx,lbx
end

kt_labor_fast (generic function with 1 method)

In [145]:
function policy_backward(;labor_prod,hh::Household,r::Float64,w::Float64,policy::policy_variables,
            tax::Tax,itp_tax,Transfer::Float64,ext_β::Array{Float64,2},p0::MarkovChain)
    
    ext_prod_n2 = kron(labor_prod.pt.state_values,ones(length(hh.β_chain.state_values)))
    ext_prod_n3 = kron(ext_prod_n2,labor_prod.vt.state_values)
    ext_prod_n1 = kron(ext_prod_n3,p0.state_values)
    trans_weight_RHS = kron(kron(kron(labor_prod.pt.markovchain,β_chain.markovchain),
                                    labor_prod.vt.weights'),labor_prod.p0.weights')
    
    Xpt,~,~,~,Yptau_r = Income_Asset_Info(a_vals = hh.a_vals,lt = kron(policy.lpt_grid,p0.state_values'),
                        wt=w*ext_prod_n1,r=r,itp_tax=itp_tax,tax=tax,Transfer=Transfer)
       
    Cpt = Xpt-policy.Kppt
    RHS = broadcast(*,(Cpt.^(-hh.σ).*Yptau_r)*trans_weight_RHS',ext_β)
    C0 = RHS.^(-1/hh.σ)
    Xt = broadcast(+,C0,hh.a_vals)
    
    return Xt,RHS
end

policy_backward (generic function with 1 method)

In [146]:
function backward_iteration(;hh::Household,rg::Array{Float64,1},Kg::Array{Float64,1},aggregate_v,policy_v,pt_path,vt_path,p0,tax_path,N_prod)
    # obtain the aggregate variable given path of (r,K)
    k_path = ((rg.+hh.δ)./hh.α).^(1/(hh.α-1))
    wg = (1-hh.α).*k_path.^(hh.α)
    Lg_path = Kg./k_path
    
    # obatin the transfers and tax policy along transitional path
    Transfer_path = zeros(T)
    Tax_schedule_path = Vector(undef,T)
    itp_tax_path = Vector(undef,T)
    for i = 1:T
        tax = Tax(tax_path.y_pshare,tax_path.y_ishare[:,i],tax_path.y_atax[:,i],
            Array{Float64,1}(undef,11), Array{Float64,1}(undef,11), Array{Float64,1}(undef,11))
        Tax_schedule_path[i],Transfer_path[i], itp_tax_path[i] = 
                        tax_policy(Tax = tax, r = rg[i], w = wg[i], k = k_path[i], L = Lg_path[i])
    end
    
    policy_path = Vector(undef,T)
    policy_path[T] = policy_v[2]
    
    # Solve policy function backward
    ext_β = kron(ones(1,length(pt_path[1].weights)),hh.β_chain.state_values')
    for i = T:-1:2
        ################################################################################
        # From Xpt in period "t+1" to Xt period "t"
        pt_chain_pt = MarkovChain(pt_path[i].markovchain_backward,pt_path[i].state_values,pt_path[i].weights)
        l_prod = productivity(pt=pt_chain_pt,vt=vt_path[i],p0=p0,β_chain=hh.β_chain)
        
        #give information and policy function at t+1, backout Xt and RHS
        Xt,RHS = policy_backward(labor_prod=l_prod, hh=hh, r=rg[i], w=wg[i], policy=policy_path[i], ext_β=ext_β,
                            tax=Tax_schedule_path[i], itp_tax=itp_tax_path[i], Transfer=Transfer_path[i],p0=p0)
        ################################################################################
        #From Xt to kt
        
        pt_chain_t = MarkovChain(pt_path[i-1].markovchain_backward,pt_path[i-1].state_values,pt_path[i-1].weights)
        l_prod_previous = productivity(pt=pt_chain_t,vt=vt_path[i-1],p0=p0,β_chain=hh.β_chain)
        
        ext_prod_n2 = kron(l_prod_previous.pt.state_values,ones(length(hh.β_chain.state_values)))
        ext_prod_n3 = kron(ext_prod_n2,l_prod_previous.vt.state_values)
        ext_prod_n1 = kron(ext_prod_n3,p0.state_values)


#         k0,lt0 = kt_labor(r=rg[i-1],w_vector=wg[i-1].*ext_prod_n1,
#             hh=hh,RHS=kron(RHS,ones(1,N_prod[4])),tax=Tax_schedule_path[i-1],
#                 itp_tax=itp_tax_path[i-1],Transfer=Transfer_path[i-1],Xt=Xt,l_prod=l_prod_previous)
        k0,lt0 = kt_labor_fast1(r=rg[i-1],w_vector=wg[i-1].*ext_prod_n1,
            hh=hh,RHS=kron(RHS,ones(1,N_prod[4])),tax=Tax_schedule_path[i-1],
                itp_tax=itp_tax_path[i-1],Transfer=Transfer_path[i-1],Xt=Xt,l_prod=l_prod_previous)
        ################################################################################
        # from kt, obtain policy function at t
        kpt0 = Array{Float64,2}(undef,hh.a_size,N_prod[2])
        for j = 1:N_prod[1]
             kpt0[:,j] = vector_linear_interpolation(k0[:,j],hh.a_vals,hh.a_vals)
        end
        lpt0 = op_labor1(r=rg[i-1],w=wg[i-1].*ext_prod_n2,hh=hh,kprime=kpt0[:,N_prod[5]*(1:N_prod[3])],
                        k_grid=repeat(a_cdf_vals,1,N_prod[3]),itp_tax=itp_tax_path[i-1],
                            Transfer=Transfer_path[i-1],tax=Tax_schedule_path[i-1])
        
#         policy_path[i-1].Xt = Xt
        policy_path[i-1].Kppt = kpt0
#         policy_path[i-1].lpt_grid = lpt0
        policy_path[i-1].Kt = k0
    end
    return policy_path
end

backward_iteration (generic function with 1 method)

In [147]:
function forward_iteration(policy_path,pt_path,vt_path)
    Gz_sum = Vector(undef,T)
    
    Gz_sum[1] = aggregate_v[1].Gz
    Lsi_store = Vector(undef,T)
    Lsi_store[1] = policy_v[1].lpti_grid
    
    Kip = zeros(T)
    Lip = zeros(T)
    Kip[1] = aggregate_v[1].K
    Lip[1] = aggregate_v[1].L
    
    for i = 2:T
        Gz_sum[i] = zeros(hh.a_cdf_size,N_prod[1])
        ext_prod_n2 = kron(pt_path[i].state_values',ones(1,length(hh.β_chain.state_values)))
        for j = 1:N_prod[1]
            Kppti[:,j] = vector_linear_interpolation(hh.a_vals, policy_path[i].Kppt[:,j],hh.a_cdf_vals)
        end
        lsi = op_labor1(r=rg[i],w=wg[i].*ext_prod_n2,hh=hh,kprime=Kppti[:,N_prod[5]*(1:N_prod[3])],
                        k_grid=repeat(hh.a_cdf_vals,1,N_prod[3]),itp_tax=itp_tax_path[i],
                            Transfer=Transfer_path[i],tax=Tax_schedule_path[i])
        
        a_cdf_t = Array{Float64,2}(undef,hh.a_cdf_size,N_prod[1])
        for j = 1:N_prod[1]
            a_cdf_t[:,j] = vector_linear_interpolation(hh.a_vals, policy_path[i].Kt[:,j],hh.a_cdf_vals)
        end
        
        Gz_ext  = kron(Gz,ones(1,l_prod.n4))
        # Obtain previous CDF by interpolation
        for j=1:l_prod.n1
            G_sz[:,j] = vector_linear_interpolation(hh.a_cdf_vals, Gz_ext[:,j],a_cdf_t[:,j])
        end
        G_sz[a_cdf_t.<hh.a_min].=0
        G_sz[a_cdf_t.>=hh.a_max].=1
        
        pt_chain = MarkovChain(pt_path[i-1].markovchain_forward,pt_path[i-1].state_values,pt_path[i-1].weights)
        l_prod = productivity(pt=pt_chain,vt=vt_path[i-1],p0=p0,β_chain=hh.β_chain)
        weight_RHS = sparse(kron(kron(kron(l_prod.pt.markovchain,β_chain.markovchain),
                                    l_prod.vt.weights'),labor_prod.p0.weights'))

        Gz = G_sz*l_prod.weight_RHS'
        Gz = broadcast(/,Gz,Gz[end,:]')
        
        # unconditional weights for persistent varible
        uncweights = kron(l_prod.pt.weights,hh.β_chain.weights)
        # obtain the aggregate capital supply
        Ks = 0.5*(hh.a_cdf_vals[1:end-1]+hh.a_cdf_vals[2:end])'*(Gz[2:end,:]-Gz[1:end-1,:])*uncweights+
        hh.a_cdf_vals[1]*Gz[1,:]'*uncweights

        Gz_ext = kron(Gz,ones(1,2))
        uncweights_vt = kron(uncweights,l_prod.vt.weights)
        Ls = sum(0.5*(lsi[1:end-1,:]+lsi[2:end,:]).*(Gz_ext[2:end,:]-Gz_ext[1:end-1,:])*uncweights_vt)+(lsi[1,:].*Gz_ext[1,:])'*uncweights_vt

        
        Gz_sum[i] = Gz
        Kip[i] = Ks
        Lip[i] = Ls
        Lsi_store[i] = lsi
    end
    rip=hh.α.*(Kip./Lip).^(hh.α-1).-hh.δ
    return rip,Kip,Lip,Lsi_store[i]
end

forward_iteration (generic function with 1 method)

# Transitional Dynamics From 1967 to Long Run

In [43]:
# Guess Transitional Period that already finishing transitioning
T = 300
# @load "/Users/mac/Dropbox/Julia/Aiyagari_EGM/tax_system.jld2" tax_policy_history
# @load "/Users/mac/Dropbox/Julia/Aiyagari_EGM/income_simulation.jld2" income_prod_list
@load "C:\\Users\\leosh\\Dropbox\\Julia\\Aiyagari_EGM\\tax_system.jld2" tax_policy_history
@load "C:\\Users\\leosh\\Dropbox\\Julia\\Aiyagari_EGM\\income_simulation.jld2" income_prod_list

# Assumption for exogenous variables: 
# (1) no change of tax policy after 2010
# (2) no change of labor income error volatility

# Extend the tax policy to period T
tax_path = tax_path_construction(tax_policy_history,T)

# Construct the transitional matrix for both pt and vt
shrink = 1.0
pt_path = ptmatrix_path_construction(income_prod_list,T,shrink)
vt_path = vt_path_construction(income_prod_list.σ_vt,T,shrink);

┌ Warning: type Main.Tax_Policy_History does not exist in workspace; reconstructing
└ @ JLD2 C:\Users\leosh\.julia\packages\JLD2\KjBIK\src\data.jl:1153
┌ Warning: type Main.income_productivity_list does not exist in workspace; reconstructing
└ @ JLD2 C:\Users\leosh\.julia\packages\JLD2\KjBIK\src\data.jl:1153


getfield(JLD2.ReconstructedTypes, Symbol("##Main.income_productivity_list#361"))(0.9733, [0.3798, 0.389548, 0.389426, 0.390336, 0.389532, 0.40699, 0.404244, 0.397245, 0.398609, 0.396507  …  0.533156, 0.538961, 0.544403, 0.549508, 0.554301, 0.558804, 0.563036, 0.567016, 0.570761, 0.574286], [0.087178, 0.122882, 0.0888819, 0.0932738, 0.0860233, 0.147986, 0.0806226, 0.0547723, 0.0969536, 0.0818535  …  0.145602, 0.145602, 0.145602, 0.145602, 0.145602, 0.145602, 0.145602, 0.145602, 0.145602, 0.145602], [0.197231, 0.146629, 0.179165, 0.178045, 0.181108, 0.221133, 0.193649, 0.221359, 0.192614, 0.2502  …  0.295296, 0.295296, 0.295296, 0.295296, 0.295296, 0.295296, 0.295296, 0.295296, 0.295296, 0.295296], [2.75, 2.79, 2.8, 2.85, 2.83, 2.79, 2.76, 2.65, 2.66, 2.62  …  1.87, 1.88, 1.8, 1.79, 1.78, 1.75, 1.79, 1.9, 1.85, 1.86])

In [104]:
pt_path = Vector(undef,T)
vt_path = Vector(undef,T)

n_tax_state, T = size(tax_policy_history.y_atax)
#Initialization
tax_path = Tax_path(tax_policy_history.y_pshare,Array{Float64,2}(undef,n_tax_state,T),
            Array{Float64,2}(undef,n_tax_state,T),Array{Float64,2}(undef,n_tax_state,T))
tax_path.y_ishare[:,1:T] .= tax_policy_history[1].y_ishare
tax_path.y_mtax[:,1:T] .= tax_policy_history[1].y_mtax
tax_path.y_atax[:,1:T] .= tax_policy_history[1].y_atax



MethodError: MethodError: no method matching getindex(::getfield(JLD2.ReconstructedTypes, Symbol("##Main.Tax_Policy_History#358")), ::Int64)

In [44]:
function op_labor2_fast(;r::Float64,w::Array{Float64,1},hh::Household,
        k_grid::Array{Float64,2},tax::Tax,RHS::Array{Float64,2},itp_tax)
"""
    Solve labor optimality condition using RHS of Euler equation for t and t+1
    be careful of the period t of RHS when using.
    """
    x1,x2 = size(k_grid)
    la = zeros(x1,x2).+0.001
    lb = ones(x1,x2)
    # labor disutility - benefit of labor supplying
    Ya = k_grid.*r+broadcast(*,la,w')
    Yatau = marginaltax(y=Ya,tax=tax)
    
    Yb = k_grid.*r+broadcast(*,lb,w')
    Ybtau = marginaltax(y=Yb,tax=tax)
    
    fla = hh.ψ*la.^hh.ζ-(-Yatau.+1).*broadcast(*,RHS,w')
    flb = hh.ψ*lb.^hh.ζ-(-Ybtau.+1).*broadcast(*,RHS,w')
    
    dlx = 0.5*(lb-la)
    lx = la+dlx
    tol=1e-6
    iter = 0
#     C_min = repeat(range(1,length=x1).*1e-6,1,x2)
    
    indexl_loop = reshape(collect(1:x1*x2),x1,x2)
    wage_loop = repeat(w',x1,1)
    lx_loop = lx
    kgrid_loop = k_grid
    RHS_loop = RHS
  
    while any(abs.(dlx).>tol)
        iter+=1
        dlx = 0.5*dlx
        Yx = kgrid_loop.*r+broadcast(*,lx_loop,wage_loop)
        Yxtau = marginaltax(y=Yx,tax=tax)
        flx = hh.ψ*lx.^hh.ζ-(-Yxtau.+1).*broadcast(*,RHS_loop,wage_loop)
        lx_loop = lx_loop -sign.(flx).*dlx
        if any(abs.(flx).<1e-8)>0
            lx[indexl_loop[abs.(flx).<1e-8]] = lx_loop[abs.(flx).<1e-8]
            indexl_loop = indexl_loop[abs.(flx).>1e-8]
            dlx = dlx[abs.(flx).>1e-8]
            wage_loop = wage_loop[abs.(flx).>1e-8]
            kgrid_loop = kgrid_loop[abs.(flx).>1e-8]
            RHS_loop = RHS_loop[abs.(flx).>1e-8]
            lx_loop = lx_loop[abs.(flx).>1e-8]
        end
    end

    lx[indexl_loop] = lx_loop
    # there are cases that FOC on labor not balanced
    # the benefit of working always surpasses the cost of working
    lx[lx.>1.0].=1.0
    lx[lx.<0.0].=0.0
    return lx
end

op_labor2_fast (generic function with 1 method)

In [45]:
function op_labor2_unit(;r::Float64,w::Float64,hh::Household,
        k_grid::Float64,tax::Tax,RHS::Float64,itp_tax)
"""
    Solve labor optimality condition using RHS of Euler equation for t and t+1
    be careful of the period t of RHS when using.
    """
    la = 0.001
    lb = 1.0
    # labor disutility - benefit of labor supplying
    Ya = k_grid*r+la*w
    if Ya>maximum(tax.y_cutoff)
        Yatau = tax.y_mtax[end]
    elseif Yx<minimum(tax.y_cutoff)
        Yatau = 0
    else
        for i =1:11
            if (Ya>tax.y_cutoff[i]) & (Ya<=tax.y_cutoff[i+1])
                Yatau = tax.y_mtax[i]
                break
            end
        end
    end
    
    Yb = k_grid*r+lb*w
    if Yb>maximum(tax.y_cutoff)
        Ybtau = tax.y_mtax[end]
    elseif Yb<minimum(tax.y_cutoff)
        Ybtau = 0
    else
        for i =1:11
            if (Yb>tax.y_cutoff[i]) & (Yb<=tax.y_cutoff[i+1])
                Ybtau = tax.y_mtax[i]
                break
            end
        end
    end
    
    fla = hh.ψ*la^hh.ζ-(-Yatau+1)*RHS*w
    flb = hh.ψ*lb^hh.ζ-(-Ybtau+1)*RHS*w
    
    dlx = 0.5*(lb-la)
    lx = la+dlx
    tol=1e-4
    iter = 0
    while any(abs(dlx)>tol)
        iter+=1
        dlx = 0.5*dlx
        Yx = k_grid*r+lx*w
        
        if Yx>maximum(tax.y_cutoff)
            Yxtau = tax.y_mtax[end]
        elseif Yx<minimum(tax.y_cutoff)
            Yxtau = 0
        else
            for i =1:11
                if (Yx>tax.y_cutoff[i]) & (Yx<=tax.y_cutoff[i+1])
                    Yxtau = tax.y_mtax[i]
                    break
                end
            end
        end
        flx = hh.ψ*lx^hh.ζ-(-Yxtau+1)*RHS*w
        lx = lx -sign(flx)*dlx
    end
    
    # there are cases that FOC on labor not balanced
    # the benefit of working always surpasses the cost of working
    if lx<0.0
        lx = 0.0
    elseif lx>1.0
        lx = 1.0
    end
    return lx
end

op_labor2_unit (generic function with 1 method)

In [178]:
rg = Array{Float64,1}(undef,T)
rg[1:T-100] = range(aggregate_v[1].r,stop = aggregate_v[2].r, length = T-100)
rg[T-100+1:end] .= aggregate_v[2].r

Kg = Array{Float64,1}(undef,T)
Kg[1:T-100] = range(aggregate_v[1].K,stop = aggregate_v[2].K, length = T-100)
Kg[T-100+1:end] .= aggregate_v[2].K

diff_r = 1
tol_r = 1e-3
relax = 0.7
it_r = 0

p0 = labor_productivity[1].p0
N_prod = [labor_productivity[1].n1, labor_productivity[1].n2, labor_productivity[1].n3,
            labor_productivity[1].n4, labor_productivity[1].n5]
# for i = 1:10
rip = zeros(T)
rip[1] = aggregate_v[1].r


    k_path = ((rg.+hh.δ)./hh.α).^(1/(hh.α-1))
    wg = (1-hh.α).*k_path.^(hh.α)
    Lg_path = Kg./k_path
    
    # obatin the transfers and tax policy along transitional path
    Transfer_path = zeros(T)
    Tax_schedule_path = Vector(undef,T)
    itp_tax_path = Vector(undef,T)
    for i = 1:T
        tax = Tax(tax_path.y_pshare,tax_path.y_ishare[:,i],tax_path.y_atax[:,i],
            Array{Float64,1}(undef,11), Array{Float64,1}(undef,11), Array{Float64,1}(undef,11))
        Tax_schedule_path[i],Transfer_path[i], itp_tax_path[i] = 
                        tax_policy(Tax = tax, r = rg[i], w = wg[i], k = k_path[i], L = Lg_path[i])
    end
    
    policy_path = Vector(undef,T)
    policy_path[T] = policy_v[2]
    
    # Solve policy function backward
    ext_β = kron(ones(1,length(pt_path[1].weights)),hh.β_chain.state_values')

    @time for i = T:-1:2
        global policy_path
        print(i,"\n")
        ################################################################################
        # From Xpt in period "t+1" to Xt period "t"
        pt_chain_pt = MarkovChain(pt_path[i].markovchain_backward,pt_path[i].state_values,pt_path[i].weights)
        l_prod = productivity(pt=pt_chain_pt,vt=vt_path[i],p0=p0,β_chain=hh.β_chain)
         
        #give information and policy function at t+1, backout Xt and RHS
        Xt,RHS = policy_backward(labor_prod=l_prod, hh=hh, r=rg[i], w=wg[i], policy=policy_path[i], ext_β=ext_β,
                            tax=Tax_schedule_path[i], itp_tax=itp_tax_path[i], Transfer=Transfer_path[i],p0=p0)
        ################################################################################
        # From Xt to kt
        
        pt_chain_t = MarkovChain(pt_path[i-1].markovchain_backward,pt_path[i-1].state_values,pt_path[i-1].weights)
        l_prod_previous = productivity(pt=pt_chain_t,vt=vt_path[i-1],p0=p0,β_chain=hh.β_chain)
        
        ext_prod_n2 = kron(l_prod_previous.pt.state_values,ones(length(hh.β_chain.state_values)))
        ext_prod_n3 = kron(ext_prod_n2,l_prod_previous.vt.state_values)
        ext_prod_n1 = kron(ext_prod_n3,p0.state_values)
        
#         RHS1 = kron(RHS,ones(1,N_prod[4]))
#         w_vector=wg[i-1].*ext_prod_n1
#         x1,x2 = size(kron(RHS,ones(1,N_prod[4])))

#         ka0 = -hh.a_max.*ones(x1,x2)
#         kb0 = 1.2.*hh.a_max.*ones(x1,x2)

#         @time la0_1 = op_labor2(r=rg[i-1],w=w_vector[2*(1:N_prod[3])],hh=hh,k_grid=ka0[:,2*(1:N_prod[3])],RHS=RHS1[:,2*(1:N_prod[3])],tax=Tax_schedule_path[i-1],itp_tax=itp_tax_path[i-1])
#         @time la0_2 = op_labor2_fast(r=rg[i-1],w=w_vector[2*(1:N_prod[3])],hh=hh,k_grid=ka0[:,2*(1:N_prod[3])],RHS=RHS1[:,2*(1:N_prod[3])],tax=Tax_schedule_path[i-1],itp_tax=itp_tax_path[i-1])

        k0,lt0 = kt_labor(r=rg[i-1],w_vector=wg[i-1].*ext_prod_n1,
            hh=hh,RHS=kron(RHS,ones(1,N_prod[4])),tax=Tax_schedule_path[i-1],
                itp_tax=itp_tax_path[i-1],Transfer=Transfer_path[i-1],Xt=Xt,l_prod=l_prod_previous)

#         @time k1 = kt_labor_fast1(r=rg[i-1],w_vector=wg[i-1].*ext_prod_n1,
#             hh=hh,RHS=kron(RHS,ones(1,N_prod[4])),tax=Tax_schedule_path[i-1],
#                 itp_tax=itp_tax_path[i-1],Transfer=Transfer_path[i-1],Xt=Xt,l_prod=l_prod_previous)
        ################################################################################
        # from kt, obtain policy function at t
        kpt0 = Array{Float64,2}(undef,hh.a_size,N_prod[1])
        for j = 1:N_prod[1]
             kpt0[:,j] = vector_linear_interpolation(k0[:,j],hh.a_vals,hh.a_vals)
        end
        lpt0 = op_labor1(r=rg[i-1],w=wg[i-1].*ext_prod_n3,hh=hh,kprime=kpt0[:,N_prod[5]*(1:N_prod[3])],
                        k_grid=repeat(hh.a_vals,1,N_prod[3]),itp_tax=itp_tax_path[i-1],
                            Transfer=Transfer_path[i-1],tax=Tax_schedule_path[i-1])
        policy = policy_variables(Xt,k0,kpt0,Array{Float64,2}(undef,hh.a_size,N_prod[3]),lpt0,
                        Array{Float64,2}(undef,hh.a_size,N_prod[3]))
        policy_path[i-1] = policy
    end
# backward_iteration(hh=hh, rg=rg, Kg=Kg, aggregate_v=aggregate_v,policy_v=policy_v,
#                     pt_path=pt_path, vt_path=vt_path,tax_path=tax_path, p0=p0, N_prod=N_prod)
#     forward_iteration
# end

300
299
298
297
296
295
294
293
292
291
290
289
288
287
286
285
284
283
282
281
280
279
278
277
276
275
274
273
272
271
270
269
268
267
266
265
264
263
262
261
260
259
258
257
256
255
254
253
252
251
250
249
248
247
246
245
244
243
242
241
240
239
238
237
236
235
234
233
232
231
230
229
228
227
226
225
224
223
222
221
220
219
218
217
216
215
214
213
212
211
210
209
208
207
206
205
204
203
202
201
200
199
198
197
196
195
194
193
192
191
190
189
188
187
186
185
184
183
182
181
180
179
178
177
176
175
174
173
172
171
170
169
168
167
166
165
164
163
162
161
160
159
158
157
156
155
154
153
152
151
150
149
148
147
146
145
144
143
142
141
140
139
138
137
136
135
134
133
132
131
130
129
128
127
126
125
124
123
122
121
120
119
118
117
116
115
114
113
112
111
110
109
108
107
106
105
104
103
102
101
100
99
98
97
96
95
94
93
92
91
90
89
88
87
86
85
84
83
82
81
80
79
78
77
76
75
74
73
72
71
70
69
68
67
66
65
64
63
62
61
60
59
58
57
56
55
54
53
52
51
50
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
3

In [179]:
@save "C:\\Users\\leosh\\Dropbox\\Julia\\Aiyagari_EGM\\backward_transition.jld2" policy_path Tax_schedule_path Transfer_path itp_tax_path


In [212]:
    Gz_sum = Vector(undef,T)
    
    Gz_sum[1] = aggregate_v[1].Gz
    Lsi_store = Vector(undef,T)
    Lsi_store[1] = policy_v[1].lpti_grid
    
    Kip = zeros(T)
    Lip = zeros(T)
    Kip[1] = aggregate_v[1].K
    Lip[1] = aggregate_v[1].L
    i = 2
    @time for i = 2:T
        print(i)
        ext_prod_n2 = kron(pt_path[i].state_values,ones(length(hh.β_chain.state_values)))
        ext_prod_n3 = kron(ext_prod_n2,vt_path[i].state_values)

        Kppti = Array{Float64,2}(undef,hh.a_cdf_size,N_prod[1])
        for j = 1:N_prod[1]
            Kppti[:,j] = vector_linear_interpolation(hh.a_vals, policy_path[i].Kppt[:,j],hh.a_cdf_vals)
        end
        lsi = op_labor1(r=rg[i],w=wg[i].*ext_prod_n3,hh=hh,kprime=Kppti[:,N_prod[5]*(1:N_prod[3])],
                        k_grid=repeat(hh.a_cdf_vals,1,N_prod[3]),itp_tax=itp_tax_path[i],
                            Transfer=Transfer_path[i],tax=Tax_schedule_path[i])
        
        a_cdf_t = Array{Float64,2}(undef,hh.a_cdf_size,N_prod[1])
        for j = 1:N_prod[1]
            a_cdf_t[:,j] = vector_linear_interpolation(hh.a_vals, policy_path[i].Kt[:,j],hh.a_cdf_vals)
        end
        
        Gz_ext  = kron(Gz_sum[i-1],ones(1,N_prod[4]))
        # Obtain previous CDF by interpolation
        G_sz = Array{Float64,2}(undef,hh.a_cdf_size,N_prod[1])
        for j=1:N_prod[1]
            G_sz[:,j] = vector_linear_interpolation(hh.a_cdf_vals, Gz_ext[:,j],a_cdf_t[:,j])
        end
        G_sz[a_cdf_t.<hh.a_min].=0
        G_sz[a_cdf_t.>=hh.a_max].=1
        
        pt_chain = MarkovChain(pt_path[i-1].markovchain_forward,pt_path[i-1].state_values,pt_path[i-1].weights)
        l_prod = productivity(pt=pt_chain,vt=vt_path[i-1],p0=p0,β_chain=hh.β_chain)
        weight_RHS = kron(kron(kron(l_prod.pt.markovchain,β_chain.markovchain),
                                    l_prod.vt.weights'),l_prod.p0.weights')

        Gz = G_sz*weight_RHS'
        Gz = broadcast(/,Gz,Gz[end,:]')
        
        # unconditional weights for persistent varible
        uncweights = kron(l_prod.pt.weights,hh.β_chain.weights)
        # obtain the aggregate capital supply
        Ks = 0.5*(hh.a_cdf_vals[1:end-1]+hh.a_cdf_vals[2:end])'*(Gz[2:end,:]-Gz[1:end-1,:])*uncweights+
        hh.a_cdf_vals[1]*Gz[1,:]'*uncweights

        Gz_ext = kron(Gz,ones(1,2))
        uncweights_vt = kron(uncweights,l_prod.vt.weights)
        Ls = sum(0.5*(lsi[1:end-1,:]+lsi[2:end,:]).*(Gz_ext[2:end,:]-Gz_ext[1:end-1,:])*uncweights_vt)+(lsi[1,:].*Gz_ext[1,:])'*uncweights_vt

        
        Gz_sum[i] = Gz
        Kip[i] = Ks
        Lip[i] = Ls
        Lsi_store[i] = lsi
    end
    rip=hh.α.*(Kip./Lip).^(hh.α-1).-hh.δ

23456789101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899100101102103104105106107108109110111112113114115116117118119120121122123124125126127128129130131132133134135136137138139140141142143144145146147148149150151152153154155156157158159160161162163164165166167168169170171172173174175176177178179180181182183184185186187188189190191192193194195196197198199200201202203204205206207208209210211212213214215216217218219220221222223224225226227228229230231232233234235236237238239240241242243244245246247248249250251252253254255256257258259260261262263264265266267268269270271272273274275276277278279280281282283284285286287288289290291292293294295296297298299300427.961359 seconds (26.16 M allocations: 278.131 GiB, 11.45% gc time)


300-element Array{Float64,1}:
 0.04239349712774375 
 0.03224249615893596 
 0.03738110614446365 
 0.04222794373890723 
 0.04618095098874028 
 0.04848000724584313 
 0.05031850233760313 
 0.05239332661541503 
 0.05458321851727499 
 0.05617241357240775 
 0.05820009248690158 
 0.05906945382749021 
 0.05972656001395314 
 ⋮                   
 0.018224385817848465
 0.018840512832816347
 0.01912277340801441 
 0.01979900239549892 
 0.02013592585192782 
 0.020810437517017777
 0.021110788879916784
 0.021099221148143332
 0.020947695060313146
 0.02069208181396963 
 0.020107450601674376
 0.019279353064495908

2147.95843142918

In [221]:
@save "C:\\Users\\leosh\\Dropbox\\Julia\\Aiyagari_EGM\\backward_transition.jld2" policy_path Tax_schedule_path Transfer_path itp_tax_path Gz_sum Kip Lip Lsi_store rip


In [168]:
i = 23
pt_chain_pt = MarkovChain(pt_path[i].markovchain_backward,pt_path[i].state_values,pt_path[i].weights)
        l_prod = productivity(pt=pt_chain_pt,vt=vt_path[i],p0=p0,β_chain=hh.β_chain)
         
        #give information and policy function at t+1, backout Xt and RHS
        Xt,RHS = policy_backward(labor_prod=l_prod, hh=hh, r=rg[i], w=wg[i], policy=policy_path[i], ext_β=ext_β,
                            tax=Tax_schedule_path[i], itp_tax=itp_tax_path[i], Transfer=Transfer_path[i],p0=p0)
        ################################################################################
        # From Xt to kt
        
        pt_chain_t = MarkovChain(pt_path[i-1].markovchain_backward,pt_path[i-1].state_values,pt_path[i-1].weights)
        l_prod_previous = productivity(pt=pt_chain_t,vt=vt_path[i-1],p0=p0,β_chain=hh.β_chain)
        
        ext_prod_n2 = kron(l_prod_previous.pt.state_values,ones(length(hh.β_chain.state_values)))
        ext_prod_n3 = kron(ext_prod_n2,l_prod_previous.vt.state_values)
        ext_prod_n1 = kron(ext_prod_n3,p0.state_values)
        
#         RHS1 = kron(RHS,ones(1,N_prod[4]))
#         w_vector=wg[i-1].*ext_prod_n1
#         x1,x2 = size(kron(RHS,ones(1,N_prod[4])))

#         ka0 = -hh.a_max.*ones(x1,x2)
#         kb0 = 1.2.*hh.a_max.*ones(x1,x2)

#         @time la0_1 = op_labor2(r=rg[i-1],w=w_vector[2*(1:N_prod[3])],hh=hh,k_grid=ka0[:,2*(1:N_prod[3])],RHS=RHS1[:,2*(1:N_prod[3])],tax=Tax_schedule_path[i-1],itp_tax=itp_tax_path[i-1])
#         @time la0_2 = op_labor2_fast(r=rg[i-1],w=w_vector[2*(1:N_prod[3])],hh=hh,k_grid=ka0[:,2*(1:N_prod[3])],RHS=RHS1[:,2*(1:N_prod[3])],tax=Tax_schedule_path[i-1],itp_tax=itp_tax_path[i-1])

        k0,lt0 = kt_labor(r=rg[i-1],w_vector=wg[i-1].*ext_prod_n1,
            hh=hh,RHS=kron(RHS,ones(1,N_prod[4])),tax=Tax_schedule_path[i-1],
                itp_tax=itp_tax_path[i-1],Transfer=Transfer_path[i-1],Xt=Xt,l_prod=l_prod_previous)
        kpt0 = Array{Float64,2}(undef,hh.a_size,N_prod[1])
        for j = 1:N_prod[1]
             kpt0[:,j] = vector_linear_interpolation(k0[:,j],hh.a_vals,hh.a_vals)
        end

UndefRefError: UndefRefError: access to undefined reference

In [169]:
policy_path[24]

UndefRefError: UndefRefError: access to undefined reference

In [158]:
k_test = k0[:,300]


    index_duplicate = findall(k_test[1:end-1].>=k_test[2:end])
 k_test[index_duplicate] = k_test[index_duplicate.+1] .-0.000001

1-element Array{Float64,1}:
 -0.9483361060879386

In [159]:
k_test

50-element Array{Float64,1}:
     -0.9483361060879386
     -0.9483351060879386
      1.8574147791383666
     13.029381826967779 
     43.082700022623044 
    106.08106664413697  
    220.45089023912112  
    408.7506094737847   
    697.5715804731664   
   1117.571064107281    
   1703.5485543086538   
   2494.3387168836043   
   3532.87826814217     
      ⋮                 
 355289.8801287061      
 394191.2024529305      
 436202.85857895704     
 481486.3800353538      
 530207.3877515614      
 582535.592061151       
 638644.7927062183      
 698712.8788319107      
 762921.8289784102      
 831457.7110807358      
 904510.682478453       
 982274.9899248476      

In [56]:
    k_path = ((rg.+hh.δ)./hh.α).^(1/(hh.α-1))
    wg = (1-hh.α).*k_path.^(hh.α)
    Lg_path = Kg./k_path
    
    # obatin the transfers and tax policy along transitional path
    Transfer_path = zeros(T)
    Tax_schedule_path = Vector(undef,T)
    itp_tax_path = Vector(undef,T)
    for i = 1:T
        tax = Tax(tax_path.y_pshare,tax_path.y_ishare[:,i],tax_path.y_atax[:,i],
            Array{Float64,1}(undef,11), Array{Float64,1}(undef,11), Array{Float64,1}(undef,11))
        Tax_schedule_path[i],Transfer_path[i], itp_tax_path[i] = 
                        tax_policy(Tax = tax, r = rg[i], w = wg[i], k = k_path[i], L = Lg_path[i])
    end
    
    policy_path = Vector(undef,T)
    policy_path[T] = policy_v[2]
    
    # Solve policy function backward
    ext_β = kron(ones(1,length(pt_path[1].weights)),hh.β_chain.state_values')
    i=T
#     for i = T:-1:2
        ################################################################################
        # From Xpt in period "t+1" to Xt period "t"
        pt_chain_pt = MarkovChain(pt_path[i].markovchain_backward,pt_path[i].state_values,pt_path[i].weights)
        l_prod = productivity(pt=pt_chain_pt,vt=vt_path[i],p0=p0,β_chain=hh.β_chain)
        
        #give information and policy function at t+1, backout Xt and RHS
        Xt,RHS = policy_backward(labor_prod=l_prod, hh=hh, r=rg[i], w=wg[i], policy=policy_path[i], ext_β=ext_β,
                            tax=Tax_schedule_path[i], itp_tax=itp_tax_path[i], Transfer=Transfer_path[i],p0=p0)
        ################################################################################
        #From Xt to kt
        
        pt_chain_t = MarkovChain(pt_path[i-1].markovchain_backward,pt_path[i-1].state_values,pt_path[i-1].weights)
        l_prod_previous = productivity(pt=pt_chain_t,vt=vt_path[i-1],p0=p0,β_chain=hh.β_chain)
        
        ext_prod_n2 = kron(l_prod_previous.pt.state_values,ones(length(hh.β_chain.state_values)))
        ext_prod_n3 = kron(ext_prod_n2,l_prod_previous.vt.state_values)
        ext_prod_n1 = kron(ext_prod_n3,p0.state_values)



1020-element Array{Float64,1}:
     0.0                 
     0.046795237174325904
     0.0                 
     0.08446813396303343 
     0.0                 
     0.046795237174325904
     0.0                 
     0.08446813396303343 
     0.0                 
     0.046795237174325904
     0.0                 
     0.08446813396303343 
     0.0                 
     ⋮                   
     0.0                 
  6474.864004752286      
     0.0                 
 11687.507386882215      
     0.0                 
  6474.864004752286      
     0.0                 
 11687.507386882215      
     0.0                 
  6474.864004752286      
     0.0                 
 11687.507386882215      

In [109]:
A = [1.0;1;1;2;3;4]

6-element Array{Float64,1}:
 1.0
 1.0
 1.0
 2.0
 3.0
 4.0

In [110]:
A[findall(A[1:end-1].==A[2:end])] = A[findall(A[1:end-1].==A[2:end])].-0.001

2-element Array{Float64,1}:
 0.999
 0.999

# Test Area

## Steady State in 1967

In [186]:
using LinearAlgebra
using Interpolations
import FastGaussQuadrature.gausshermite
using Distributions
using SparseArrays
# Firm parameter
α = 0.36 #capital share
δ = 0.048 #depreciation rate

# Consumer parameter
σ = 1.5
β_μ = 0.92
ψ = 27.0
ζ = 2.0

# government parameter
λT = 0.6 #fraction of revenue distribution as transfer

# stochastic discounting
β_markovchain,β_state_values,β_weight = GaussHermite_matrix(n=11,rho=0.992,
                                        σ=sqrt(0.0021^2/(1-0.992^2)),μ=β_μ,exponential=0)
β_chain = MarkovChain(β_markovchain,β_state_values,β_weight)

##############
a_max = 1e6
a_min = 1e-10
# a_min = -0.24
a_size = 30
a_cdf_size = 200

# Generate an instance of households with grids, utility setup
hh = parameters(σ=σ,β_chain=β_chain,α=α,δ=δ,a_min=a_min,a_max=a_max,
            ψ=ψ,ζ=ζ, a_size=a_size,a_cdf_size=a_cdf_size,λT=λT)

###############
# Persistent productivity
shrink = 1.6
# Np = 17
n = 15 # number of Gausss-Hermite nodes
ρ_pt = 0.9733
σ_pt = 0.3798
kt = 2.75

pt_chain = ptmatrix(n=15, ρ_pt=0.9733,σ_pt=0.3798,kt=2.75,shrink=1)

###############
# Transitory productivity
σ_vt = 0.1972
Nv = 2
v_nodes,v_weight = gausshermite(Nv)
v_weight = v_weight./sqrt(pi)
v_grid = sqrt(2)*σ_vt*v_nodes
v_grid = exp.(v_grid)./exp(σ_vt^2/2)

vt_chain = MarkovChain(repeat(v_weight',length(v_grid),1),v_grid,v_weight)
###############
# zero earning state
p0 = MarkovChain([0.075 0.925;0.075 0.925],[0,1],[0.075,0.925])

##############
# group all productivity into one struct and a related transitional matrix
l_prod = productivity(pt=pt_chain,vt=vt_chain,p0=p0,β_chain=β_chain)

# # Initialize policy function
# l_prod.n1: all size
# l_prod.n2: size of persistent variables
# l_prod.n3: size exclude 0 earings state

#(Xt,Kt,Kppt,lt_egm,lt_grid,lpt_grid)
policy = policy_variables(Array{Float64,2}(undef,hh.a_size,l_prod.n2),
        Array{Float64,2}(undef,hh.a_size,l_prod.n1),
        Array{Float64,2}(undef,hh.a_size,l_prod.n1),
        Array{Float64,2}(undef,hh.a_size,l_prod.n3),
        Array{Float64,2}(undef,hh.a_size,l_prod.n3),
        Array{Float64,2}(undef,hh.a_size,l_prod.n3),
        Array{Float64,2}(undef,hh.a_size,l_prod.n3))

# Initialize tax system
y_pshare = [0.2,0.2,0.2,0.2,0.1,0.05,0.04,0.005,0.004,0.0009,0.0001]
y_ishare = [2.21,7.66,15.41,23.97,16.42,10.60,13.06,3.20,4.26,2.26,0.96]./100
y_atax=[15.1,17.2,18.7,19.0,19.0,20.0,24.5,34.1,41.4,52.7,64.4]./100

#(yp_share,yi_share,y_atax,y_mtax,y_cutoff,y_end_tax)
tax = Tax(y_pshare,y_ishare,y_atax,Array{Float64,1}(undef,11),Array{Float64,1}(undef,11),
        Array{Float64,1}(undef,11))

aggregate = aggregate_variables(0.0,0.0,0.0,0.0,0.0,
        tax,Array{Float64,2}(undef,hh.a_size,l_prod.n2))

error_compute = error_recording(0.0,0.0,0.0,0.0,0.0,0.0,0.0)

# @time fr,policy,aggregate,error = bisection(rmin=0.001,rmax=0.08,
#     f=partial_equilibrium,hh=hh,tax=tax,l_prod=l_prod);
@time fx,~,aggregate= partial_equilibrium(r=0.03,hh=hh,policy=policy,firsttime=1,
    tax=tax,aggregate=aggregate,l_prod=l_prod,error=error_compute);

current labor supply: 0.0
 37.995357 seconds (71.50 M allocations: 15.104 GiB, 5.53% gc time)


In [232]:
using LinearAlgebra
using Interpolations
import FastGaussQuadrature.gausshermite
using Distributions
using SparseArrays
# Firm parameter
α = 0.36 #capital share
δ = 0.048 #depreciation rate

# Consumer parameter
σ = 1.5
β_μ = 0.92
ψ = 27.0
ζ = 2.0

# government parameter
λT = 0.6 #fraction of revenue distribution as transfer

# stochastic discounting
β_markovchain,β_state_values,β_weight = GaussHermite_matrix(n=11,rho=0.992,
                                        σ=sqrt(0.0021^2/(1-0.992^2)),μ=β_μ,exponential=0)
β_chain = MarkovChain(β_markovchain,β_state_values,β_weight)

##############
a_max = 1e6
a_min = 1e-10
# a_min = -0.24
a_size = 50
a_cdf_size = 70

# Generate an instance of households with grids, utility setup
hh = parameters(σ=σ,β_chain=β_chain,α=α,δ=δ,a_min=a_min,a_max=a_max,
            ψ=ψ,ζ=ζ, a_size=a_size,a_cdf_size=a_cdf_size,λT=λT)

###############
# Persistent productivity
shrink = 1.6
# Np = 17
n = 15 # number of Gausss-Hermite nodes
ρ_pt = 0.9733
σ_pt = 0.3798
kt = 2.75

pt_chain = ptmatrix(n=15, ρ_pt=0.9733,σ_pt=0.3798,kt=2.75,shrink=1)

###############
# Transitory productivity
σ_vt = 0.1972
Nv = 2
v_nodes,v_weight = gausshermite(Nv)
v_weight = v_weight./sqrt(pi)
v_grid = sqrt(2)*σ_vt*v_nodes
v_grid = exp.(v_grid)./exp(σ_vt^2/2)

vt_chain = MarkovChain(repeat(v_weight',length(v_grid),1),v_grid,v_weight)
###############
# zero earning state
p0 = MarkovChain([0.075 0.925;0.075 0.925],[0,1],[0.075,0.925])

##############
# group all productivity into one struct and a related transitional matrix
l_prod = productivity(pt=pt_chain,vt=vt_chain,p0=p0,β_chain=β_chain)

# # Initialize policy function
# l_prod.n1: all size
# l_prod.n2: size of persistent variables
# l_prod.n3: size exclude 0 earings state

#(Xt,Kt,Kppt,lt_egm,lt_grid,lpt_grid)
policy = policy_variables(Array{Float64,2}(undef,hh.a_size,l_prod.n2),
        Array{Float64,2}(undef,hh.a_size,l_prod.n1),
        Array{Float64,2}(undef,hh.a_size,l_prod.n1),
        Array{Float64,2}(undef,hh.a_size,l_prod.n3),
        Array{Float64,2}(undef,hh.a_size,l_prod.n3),
        Array{Float64,2}(undef,hh.a_size,l_prod.n3),
        Array{Float64,2}(undef,hh.a_size,l_prod.n3))

# Initialize tax system
y_pshare = [0.2,0.2,0.2,0.2,0.1,0.05,0.04,0.005,0.004,0.0009,0.0001]
y_ishare = [2.21,7.66,15.41,23.97,16.42,10.60,13.06,3.20,4.26,2.26,0.96]./100
y_atax=[15.1,17.2,18.7,19.0,19.0,20.0,24.5,34.1,41.4,52.7,64.4]./100

#(yp_share,yi_share,y_atax,y_mtax,y_cutoff,y_end_tax)

aggregate = aggregate_variables(0.0,0.0,0.0,0.0,0.0,
        tax,Array{Float64,2}(undef,hh.a_size,l_prod.n2))

error_compute = error_recording(0.0,0.0,0.0,0.0,0.0,0.0,0.0)

r = 0.03
firsttime = 1
k = ((r+hh.δ)/hh.α)^(1/(hh.α-1))
w = (1-hh.α)*k^hh.α

if firsttime==1
#     Ls=0.3
    Ls=0.30303221006144004
else
    Ls = aggregate.L
end

tol_Ls = 1e-3
diff_Ls = tol_Ls+1
iter_Ls = 0
maxiter_Ls = 10
tax = Tax(y_pshare,y_ishare,y_atax,Array{Float64,1}(undef,11),Array{Float64,1}(undef,11),
        Array{Float64,1}(undef,11))


while (diff_Ls>tol_Ls)&&(iter_Ls<maxiter_Ls)
    iter_Ls+=1
    if iter_Ls>1
        firsttime = 0
    end
    print(Ls, "\n")
    # Solve policy function using Endogenous Grid Method
    policy1,tax1,itp_tax1,Transfer = policy_EGM(r=r,w=w,hh=hh,policy=policy,
                                firsttime=firsttime,l_prod=l_prod,tax=tax,Ls=Ls,
                                aggregate=aggregate,error=error)
    policy = policy1
    tax = tax1
    # Solve for distribution given policy function
    Gz,K,Ls1,lpti_grid = cdf_asset(r=r,w=w,hh=hh,policy=policy,firsttime=firsttime,
        itp_tax=itp_tax1,Transfer=Transfer,aggregate=aggregate,l_prod=l_prod,tax=tax);
    
    aggregate.K = K
    aggregate.Gz = Gz
    policy.lpti_grid = lpti_grid
    diff_Ls = norm(Ls1-Ls)
        println(iter_Ls," diff in Ls: ",diff_Ls," Current Ls: ",Ls1)
    if (iter_Ls>=maxiter_Ls)||(diff_Ls<=tol_Ls)
        Ls = Ls1
        aggregate.L = Ls1
        print("yes")
        break
    end
    aggregate.L = Ls1
    Ls = 0.3*Ls1+0.7*Ls
end
# obtain the implied demand side interest rate given Ks and Ls
rs = hh.α*(aggregate.K/aggregate.L)^(hh.α-1)-hh.δ
fr = rs-r
println("current labor supply: ",aggregate.L)
aggregate.r = rs
aggregate.w = (1-hh.α)*(aggregate.K/aggregate.L)^(hh.α)

0.30303221006144004
1 diff in Ls: 0.007075156810026739 Current Ls: 0.3101073668714668
0.30515475710444806
2 diff in Ls: 0.003915865751766223 Current Ls: 0.3090706228562143
0.3063295168299779
3 diff in Ls: 0.002804519794066651 Current Ls: 0.30913403662404454
0.3071708727681979
4 diff in Ls: 0.001911333871411347 Current Ls: 0.3090822066396092
0.30774427292962125
5 diff in Ls: 0.0013402392475265756 Current Ls: 0.30908451217714783
0.3081463447038792
6 diff in Ls: 0.0009363013493315364 Current Ls: 0.30908264605321073
yescurrent labor supply: 0.30908264605321073


1.575253314602262

In [233]:
aggregate.K

3.772759698232649

### obtaining the inequality measure for both capital and income

In [ ]:
Ksh50,Ksh10,Ksh1,Ksh01,Ksh001,Ng_k = wealth_inequality(;hh=hh,l_prod=l_prod,K=aggregate.K,Gz=aggregate.Gz)
Ysh50,Ysh10,Ysh1,Ysh01,Ysh001= income_inequality(hh=hh,l_prod=l_prod,r=aggregate.r,
                                            w=aggregate.w,K=aggregate.K,Lpti=policy.lpti_grid,k_dist=aggregate.Gz)

## Partial Equilibrium given interest rate

In [49]:
# Main File
# solve equilibrium interest rate for standard Aiyagari model

using LinearAlgebra
using Interpolations
import FastGaussQuadrature.gausshermite
using Distributions
using SparseArrays
# Firm parameter
α = 0.36 #capital share
δ = 0.048 #depreciation rate

# Consumer parameter
σ = 1.5
β_μ = 0.92
ψ = 27.0
ζ = 2.0

# government parameter
λT = 0.6 #fraction of revenue distribution as transfer

# stochastic discounting

β_markovchain,β_state_values,β_weight = GaussHermite_matrix(n=11,rho=0.992,
                                        σ=sqrt(0.0019^2/(1-0.992^2)),μ=β_μ,exponential=0)
β_chain = MarkovChain(β_markovchain,β_state_values,β_weight)

###############
# Persistent productivity
shrink = 1.6
# Np = 17
n = 15 # number of Gausss-Hermite nodes
ρ_pt = 0.9733
σ_pt = 0.3798
kt = 2.75

pt_chain = ptmatrix(n=15, ρ_pt=0.9733,σ_pt=0.3798,kt=2.75,shrink=1)

###############
# Transitory productivity
σ_vt = 0.1972
Nv = 2
v_nodes,v_weight = gausshermite(Nv)
v_weight = v_weight./sqrt(pi)
v_grid = sqrt(2)*σ_vt*v_nodes
v_grid = exp.(v_grid)./exp(σ_vt^2/2)

vt_chain = MarkovChain(repeat(v_weight',length(v_grid),1),v_grid,v_weight)
###############
# zero earning state
p0 = MarkovChain([0.075 0.925;0.075 0.925],[0,1],[0.075,0.925])

##############
# group all productivity into one struct and a related transitional matrix
l_prod = productivity(pt=pt_chain,vt=vt_chain,p0=p0,β_chain=β_chain)

##############
a_max = 1e6
# a_min = 1e-10
a_min = -0.24
a_size = 50
a_cdf_size = 70

# # Generate an instance of households with grids, utility setup
hh = parameters(σ=σ,β_chain=β_chain,α=α,δ=δ,a_min=a_min,a_max=a_max,
            ψ=ψ,ζ=ζ, a_size=a_size,a_cdf_size=a_cdf_size,λT=λT)

# # Initialize policy function
# l_prod.n1: all size
# l_prod.n2: size of persistent variables
# l_prod.n3: size exclude 0 earings state

#(Xt,Kt,Kppt,lt_egm,lt_grid,lpt_grid)
policy = policy_variables(Array{Float64,2}(undef,hh.a_size,l_prod.n2),
                          Array{Float64,2}(undef,hh.a_size,l_prod.n1),
                          Array{Float64,2}(undef,hh.a_size,l_prod.n1),
                          Array{Float64,2}(undef,hh.a_size,l_prod.n3),
                          Array{Float64,2}(undef,hh.a_size,l_prod.n3),
                          Array{Float64,2}(undef,hh.a_size,l_prod.n3))

# # Initialize tax system
y_pshare = [0.2,0.2,0.2,0.2,0.1,0.05,0.04,0.005,0.004,0.0009,0.0001]
y_ishare = [2.21,7.66,15.41,23.97,16.42,10.60,13.06,3.20,4.26,2.26,0.96]./100
y_atax=[15.1,17.2,18.7,19.0,19.0,20.0,24.5,34.1,41.4,52.7,64.4]./100

#(yp_share,yi_share,y_atax,y_mtax,y_cutoff,y_end_tax)
tax = Tax(y_pshare,y_ishare,y_atax,Array{Float64,1}(undef,11),Array{Float64,1}(undef,11),
        Array{Float64,1}(undef,11))

aggregate = aggregate_variables(0.0,0.0,0.0,0.0,0.0,
        tax,Array{Float64,2}(undef,hh.a_size,l_prod.n2))

error = error_recording(0.0,0.0,0.0,0.0,0.0,0.0,0.0)

    r = 0.03    
    k = ((r+hh.δ)/hh.α)^(1/(hh.α-1))
    w = (1-hh.α)*k^hh.α
    firsttime =1
    if firsttime==1
        Ls=0.3
    else
        Ls = aggregate.L
    end

    tol_Ls = 1e-3
    diff_Ls = tol_Ls+1
    iter_Ls = 0
    maxiter_Ls = 10

#     while (diff_Ls>tol_Ls)&&(iter_Ls<maxiter_Ls)
#         global policy,tax,error
        iter_Ls+=1
        if iter_Ls>1
            firsttime = 0
        end
#         print(firsttime)
        # Solve policy function using Endogenous Grid Method
    tol_x = 1e-5
    tol_l = 1e-5
    diff_l = tol_l+1
    iter_l = 0
    maxiter_x=2000
    maxiter_l=100

    tol_Ls = 1e-3
    diff_Ls = tol_Ls+1
    iter_Ls = 0
    maxiter_Ls = 5
    
    # Solve policy function using Endogenous Grid Method
    @time policy,tax,itp_tax,Transfer = policy_EGM(r=r,w=w,hh=hh,policy=policy,
                                firsttime=firsttime,l_prod=l_prod,tax=tax,Ls=Ls,error=error)

# policy_EGM(;hh::Household,policy::policy_variables,
#         firsttime::Int64,tax::Tax,r::Float64,w::Float64,Ls::Float64,
#         l_prod::productivity_variables,error::error_recording)
    #initialize CDF
    Gz = (repeat(hh.a_cdf_vals,1,l_prod.n2).-minimum(hh.a_cdf_vals))./
                            (maximum(hh.a_cdf_vals)-minimum(hh.a_cdf_vals))

    # # Solve for distribution given policy function
    @time Gz,K,Ls1,lpti_grid = cdf_asset(r=r, w=w, hh=hh,policy=policy,
        itp_tax=itp_tax, Transfer=Transfer, l_prod=l_prod, tax=tax, Gz=Gz);


 22.438016 seconds (49.38 M allocations: 10.228 GiB, 6.68% gc time)
  5.825292 seconds (20.25 M allocations: 2.482 GiB, 4.92% gc time)


In [72]:
marginaltax(y=[1],tax=Tax_schedule_path[T])

BoundsError: BoundsError: attempt to access (1,)
  at index [2]

In [75]:
y = [1]
a,b = size(y)
    
    ymt = zeros(a,b)
    ymt[y.>maximum(tax.y_cutoff)].=tax.y_mtax[end]
    ymt[y.<minimum(tax.y_cutoff)].=0.0
    
#     for i = 1:11
#         Indi = (y.>tax.y_cutoff[i]).*(y.<=tax.y_cutoff[i+1])
#         print(Indi)
#         ymt[Indi] .= tax.y_mtax[i]
#     end

BoundsError: BoundsError: attempt to access (1,)
  at index [2]

In [80]:
y'

1×1 Adjoint{Int64,Array{Int64,1}}:
 1

In [160]:
A=[1;2;3;4;5]

5-element Array{Int64,1}:
 1
 2
 3
 4
 5

In [162]:
asdf = findall(A[1:end-1].>A[2:end])

0-element Array{Int64,1}

In [164]:
asdf.+1

0-element Array{Int64,1}